In [169]:
!pip install torch transformers accelerate pandas numpy nltk scikit-learn reportlab PyPDF2 pdfplumber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [170]:
import os
import json
import pandas as pd
from typing import Dict, List, Any, Optional
try:
    from typing import Tuple
except ImportError:
    # For older Python versions
    Tuple = tuple
from dataclasses import dataclass, field
from datetime import datetime
import re
from collections import Counter, defaultdict
import numpy as np
import torch

# PDF generation
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak, KeepTogether
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_LEFT, TA_JUSTIFY, TA_RIGHT

# Natural Language Processing
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

# PDF reading
import PyPDF2
import pdfplumber

# HuggingFace Transformers for LLM integration
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import warnings
warnings.filterwarnings("ignore")

In [171]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('vader_lexicon')
except LookupError:
    nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [172]:
@dataclass
class ModuleMapping:
    module_name: str = ""
    module_code: str = ""
    required: bool = False
    available: bool = False
    coverage_percentage: float = 0.0
    status: str = "Unknown"
    notes: str = ""
    client_priority: str = "Medium"
    description: str = ""
    components: str = ""
    strengths: str = ""
    alignment: str = ""

@dataclass
class GapItem:
    gap_type: str = "Unknown"
    description: str = ""
    severity: str = "Medium"
    business_impact: str = ""
    mitigation_strategy: str = ""
    effort_required: str = "2-4 weeks"
    cost_estimate: str = "$10K-$25K"
    dependencies: List[str] = field(default_factory=list)
    risk_if_unaddressed: str = ""

@dataclass
class WinLossFactor:
    factor_type: str = "Unknown"  # Win or Loss
    description: str = ""
    impact: str = "Medium"
    confidence: float = 0.85
    category: str = "General"

@dataclass
class Requirement:
    requirement_type: str = "Functional"
    description: str = ""
    priority: str = "Should Have"
    our_capability: str = "Partial"
    effort_to_meet: str = "2-4 weeks"

@dataclass
class RiskAssessment:
    risk_type: str = "Technical"
    description: str = ""
    probability: str = "Medium"
    impact: str = "Medium"
    mitigation_plan: str = ""

@dataclass
class RFPFeasibilityAssessment:
    can_respond: bool
    confidence_score: float
    win_probability: float
    total_modules: int
    available_modules: int
    partial_modules: int
    missing_modules: int
    critical_gaps: List[GapItem]
    all_gaps: List[GapItem]
    strengths: List[str]
    weaknesses: List[str]
    opportunities: List[str]
    threats: List[str]
    required_actions: List[tuple]
    risks: List[RiskAssessment]
    investment_required: str
    timeline_estimate: str
    resource_requirements: str
    executive_summary: str = ""
    detailed_rationale: str = ""
    competitive_positioning: str = ""

In [173]:
@dataclass
class TechnicalProposal:
    solution_overview: str = ""
    technical_architecture: str = ""
    implementation_approach: str = ""
    integration_strategy: str = ""
    security_framework: str = ""
    performance_specifications: str = ""
    testing_methodology: str = ""
    deployment_strategy: str = ""
    
@dataclass
class FinancialProposal:
    total_cost: str = "$0"
    cost_breakdown: Dict[str, str] = field(default_factory=dict)
    payment_schedule: str = ""
    roi_analysis: str = ""
    cost_justification: str = ""
    pricing_model: str = ""
    maintenance_costs: str = ""
    optional_services: Dict[str, str] = field(default_factory=dict)

@dataclass
class RFPResponse:
    executive_summary: str = ""
    technical_proposal: TechnicalProposal = field(default_factory=TechnicalProposal)
    financial_proposal: FinancialProposal = field(default_factory=FinancialProposal)
    project_timeline: str = ""
    risk_mitigation: str = ""
    team_composition: str = ""
    compliance_statement: str = ""
    references_case_studies: str = ""
    assumptions_dependencies: str = ""


In [174]:
class LLMProcessor:
    def __init__(self):
        self.model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        try:
            from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
            import torch
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(self.model_name)
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.model.to(self.device)
            self.pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                device=0 if self.device == "cuda" else -1
            )
            print(f"Initialized LLMProcessor with {self.model_name} on {self.device}")
        except Exception as e:
            print(f"Error initializing TinyLlama: {e}")
            raise RuntimeError("Failed to initialize LLMProcessor")

    def _generate_text(self, prompt: str, max_length: int = 500) -> str:
        try:
            # Encode input to check token count
            input_tokens = self.tokenizer.encode(prompt, return_tensors="pt")
            input_length = len(input_tokens[0])
            
            # Set reasonable limits
            max_input_tokens = 800  # Reduced from 1500
            min_output_tokens = 50   # Minimum tokens to generate
            
            # Truncate input if too long
            if input_length > max_input_tokens:
                truncated_tokens = input_tokens[0][:max_input_tokens]
                prompt = self.tokenizer.decode(truncated_tokens, skip_special_tokens=True)
                input_length = max_input_tokens
                print(f"Input prompt truncated to {input_length} tokens")
            
            # Calculate available tokens for generation
            max_new_tokens = max(min_output_tokens, max_length - input_length)
            
            # If still not enough tokens, use a shorter prompt
            if max_new_tokens < min_output_tokens:
                # Take only the last part of the prompt to keep context
                words = prompt.split()
                shortened_prompt = " ".join(words[-100:])  # Last 100 words
                input_tokens = self.tokenizer.encode(shortened_prompt, return_tensors="pt")
                input_length = len(input_tokens[0])
                max_new_tokens = max(min_output_tokens, max_length - input_length)
                prompt = shortened_prompt
                print(f"Prompt further shortened to {input_length} tokens")
            
            # Ensure we have positive tokens for generation
            if max_new_tokens <= 0:
                print(f"Cannot generate text: insufficient tokens. Input: {input_length}, Available: {max_new_tokens}")
                return ""
            
            outputs = self.pipeline(
                prompt,
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                return_full_text=False,
                pad_token_id=self.tokenizer.eos_token_id,
                no_repeat_ngram_size=3
                # Removed early_stopping - not valid for this model
            )[0]["generated_text"].strip()
            
            # Clean up output
            outputs = re.sub(r'^You are an expert.*?[\n\s]*', '', outputs, flags=re.DOTALL)
            return outputs
            
        except Exception as e:
            print(f"Error generating text: {e}")
            return ""

    def analyze_module_coverage(self, modules: List[ModuleMapping]) -> str:
        module_summary = "\n".join([f"- {m.module_name}: {m.status} (Coverage: {m.coverage_percentage}%)" for m in modules])
        prompt = f"""
        You are an expert RFP analyst. Analyze the following module coverage data:
        {module_summary}
        Provide a concise analysis (max 100 words) of the module coverage, highlighting strengths and weaknesses.
        """
        return self._generate_text(prompt, max_length=200) or "Module coverage analysis unavailable."

    def analyze_gaps_impact(self, gaps: List[GapItem]) -> str:
        gap_summary = "\n".join([f"- {g.description} (Type: {g.gap_type}, Severity: {g.severity})" for g in gaps])
        prompt = f"""
        You are an expert RFP analyst. Analyze the following gaps:
        {gap_summary}
        Provide a concise analysis (max 100 words) of the impact of these gaps, focusing on critical gaps and mitigation potential.
        """
        return self._generate_text(prompt, max_length=200) or "Gap impact analysis unavailable."

    def analyze_win_loss_patterns(self, win_loss_factors: List[WinLossFactor]) -> str:
        factor_summary = "\n".join([f"- {f.description} (Type: {f.factor_type}, Impact: {f.impact})" for f in win_loss_factors])
        prompt = f"""
        You are an expert RFP analyst. Analyze the following win/loss factors:
        {factor_summary}
        Provide a concise analysis (max 100 words) of patterns in wins and losses, identifying opportunities.
        """
        return self._generate_text(prompt, max_length=200) or "Win/loss pattern analysis unavailable."

    def generate_executive_recommendation(self, assessment: RFPFeasibilityAssessment) -> str:
        prompt = f"""
        You are an expert RFP analyst. Based on the following assessment:
        - Can Respond: {assessment.can_respond}
        - Confidence Score: {assessment.confidence_score:.1f}%
        - Win Probability: {assessment.win_probability:.1f}%
        - Module Coverage: {assessment.available_modules}/{assessment.total_modules}
        - Critical Gaps: {len(assessment.critical_gaps)}
        Provide a concise executive recommendation (max 100 words) for whether to proceed with the RFP response.
        """
        return self._generate_text(prompt, max_length=200) or "Recommendation: Proceed if gaps can be addressed."

    def analyze_competitive_positioning(self, modules: List[ModuleMapping], gaps: List[GapItem], win_loss_factors: List[WinLossFactor]) -> str:
        module_summary = "\n".join([f"- {m.module_name}: {m.status}" for m in modules])
        gap_summary = "\n".join([f"- {g.description} (Severity: {g.severity})" for g in gaps])
        factor_summary = "\n".join([f"- {f.description} (Type: {f.factor_type})" for f in win_loss_factors])
        prompt = f"""
        You are an expert RFP analyst. Analyze the competitive positioning based on:
        Modules:
        {module_summary}
        Gaps:
        {gap_summary}
        Win/Loss Factors:
        {factor_summary}
        Provide a concise competitive analysis (max 150 words).
        """
        return self._generate_text(prompt, max_length=300) or "Competitive positioning analysis unavailable."

    def generate_action_plan(self, gaps: List[GapItem], missing_modules: List[ModuleMapping], timeline: str, investment: str) -> str:
        gap_summary = "\n".join([f"- {g.description} (Severity: {g.severity})" for g in gaps])
        module_summary = "\n".join([f"- {m.module_name}" for m in missing_modules])
        prompt = f"""
        You are an expert RFP analyst. Generate an action plan to address:
        Gaps:
        {gap_summary}
        Missing Modules:
        {module_summary}
        Timeline: {timeline}
        Investment: {investment}
        Provide a concise action plan (max 150 words).
        """
        return self._generate_text(prompt, max_length=300) or "Action plan: Address critical gaps and develop missing modules."

In [175]:
class DocumentProcessor:
    def __init__(self):
        self.llm_processor = LLMProcessor()  
        
    def extract_text_from_pdf(self, file_path: str) -> str:
        """Enhanced PDF text extraction with multiple fallback methods"""
        try:
            # Primary method: pdfplumber
            import pdfplumber
            with pdfplumber.open(file_path) as pdf:
                text = ""
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                
                # If pdfplumber fails, try PyPDF2 as fallback
                if not text.strip():
                    import PyPDF2
                    with open(file_path, 'rb') as file:
                        pdf_reader = PyPDF2.PdfReader(file)
                        for page in pdf_reader.pages:
                            text += page.extract_text() + "\n"
                
                return text.strip()
        except Exception as e:
            print(f"Error extracting text from {file_path}: {e}")
            return ""

    def parse_modules_enhanced(self, text: str) -> List[ModuleMapping]:
        """Significantly improved module parsing with advanced pattern recognition"""
        modules = []
        
        # Strategy 1: Advanced structured format parsing
        modules.extend(self._parse_structured_modules_v2(text))
        
        # Strategy 2: Table-based module extraction
        if not modules:
            modules.extend(self._parse_table_modules(text))
        
        # Strategy 3: Section-based module parsing
        if len(modules) < 3:
            modules.extend(self._parse_section_modules(text))
        
        # Strategy 4: Context-aware module detection
        if len(modules) < 2:
            modules.extend(self._parse_context_modules(text))
        
        # Strategy 5: NLP-based module extraction
        if len(modules) < 1:
            modules.extend(self._parse_nlp_modules(text))
        
        # Remove duplicates and validate
        modules = self._deduplicate_modules(modules)
        
        # Enhance with LLM analysis
        if self.llm_processor and modules:
            modules = self._enhance_modules_with_llm_v2(modules, text)
        
        return modules if modules else self._create_default_modules()
    
    def _parse_structured_modules_v2(self, text: str) -> List[ModuleMapping]:
        """Advanced structured module parsing with multiple patterns"""
        modules = []
        
        # Enhanced patterns for different document formats
        patterns = [
            # Pattern 1: Full module specification
            r'(?:Module|Component|System)\s*(?:#|:)?\s*([^:\n]+?)(?:\s*[-–—]\s*)?(?:Description|Overview|Details?):\s*(.*?)(?:[-–—]\s*(?:Components?|Features?|Capabilities?):\s*(.*?))?(?:[-–—]\s*(?:Status|Alignment|Coverage?):\s*(.*?))?(?=(?:Module|Component|System)|$)',
            
            # Pattern 2: Bullet-style modules with details
            r'[•\-\*]\s*([^:\n]+?)\s*(?:Module|System|Component)[:\s]*(.*?)(?=\n[•\-\*]|\n\n|$)',
            
            # Pattern 3: Numbered modules
            r'(\d+\.?\s+)([^:\n]+?)(?:\s*[-–—]\s*)?(.*?)(?=\d+\.|\n\n|$)',
            
            # Pattern 4: Header-style modules
            r'^([A-Z][A-Za-z\s]+(?:Module|System|Component|Service))\s*\n(.*?)(?=^[A-Z][A-Za-z\s]+(?:Module|System|Component|Service)|\Z)',
            
            # Pattern 5: Table-row style
            r'([A-Za-z\s]+(?:Module|System|Component))\s*\|\s*([^|]+)\s*\|\s*([^|]+)\s*\|\s*([^|\n]+)'
        ]
        
        for pattern in patterns:
            matches = re.findall(pattern, text, re.MULTILINE | re.DOTALL | re.IGNORECASE)
            
            for match in matches:
                if isinstance(match, tuple) and len(match) >= 2:
                    module_name = self._clean_module_name(match[0] if not match[0].isdigit() else match[1])
                    description = self._clean_description(match[1] if not match[0].isdigit() else match[2] if len(match) > 2 else "")
                    
                    if len(module_name) > 3 and len(description) > 20:
                        # Enhanced status determination
                        status, coverage = self._advanced_status_analysis(description, text)
                        priority = self._advanced_priority_analysis(module_name, description)
                        
                        modules.append(ModuleMapping(
                            module_name=module_name,
                            module_code=self._generate_module_code(module_name),
                            status=status,
                            coverage_percentage=coverage,
                            client_priority=priority,
                            description=description[:500],  # Limit description length
                            components=self._extract_components(description),
                            strengths=self._extract_strengths(description),
                            alignment=self._extract_alignment(description),
                            notes=f"Advanced structured parsing - Pattern matched",
                            required=True,
                            available=status in ["Available", "Partial"]
                        ))
        
        return modules
    
    def _parse_table_modules(self, text: str) -> List[ModuleMapping]:
        """Parse modules from table-like structures"""
        modules = []
        
        # Look for table headers and data
        table_patterns = [
            r'Module\s*\|\s*Status\s*\|\s*Description.*?\n((?:.*?\|.*?\|.*?\n?)+)',
            r'Component\s*\|\s*Coverage\s*\|\s*Notes.*?\n((?:.*?\|.*?\|.*?\n?)+)',
            r'System\s*Status\s*Description.*?\n((?:.*?\s+.*?\s+.*?\n?)+)'
        ]
        
        for pattern in table_patterns:
            match = re.search(pattern, text, re.MULTILINE | re.IGNORECASE)
            if match:
                table_data = match.group(1)
                rows = [row.strip() for row in table_data.split('\n') if row.strip()]
                
                for row in rows:
                    if '|' in row:
                        parts = [part.strip() for part in row.split('|')]
                        if len(parts) >= 3:
                            module_name = self._clean_module_name(parts[0])
                            status_text = parts[1] if len(parts) > 1 else ""
                            description = parts[2] if len(parts) > 2 else ""
                            
                            if len(module_name) > 3:
                                status, coverage = self._parse_status_from_text(status_text)
                                
                                modules.append(ModuleMapping(
                                    module_name=module_name,
                                    module_code=self._generate_module_code(module_name),
                                    status=status,
                                    coverage_percentage=coverage,
                                    description=description,
                                    notes="Extracted from table structure",
                                    required=True,
                                    available=status in ["Available", "Partial"]
                                ))
        
        return modules
    
    def _parse_section_modules(self, text: str) -> List[ModuleMapping]:
        """Parse modules from document sections"""
        modules = []
        
        # Split text into sections based on common delimiters
        sections = re.split(r'\n(?=\d+\.|\n[A-Z][A-Za-z\s]+:|\n#{1,3}\s)', text)
        
        for section in sections:
            if len(section) > 100:  # Substantial section
                # Look for module indicators
                if any(indicator in section.lower() for indicator in ['module', 'component', 'system', 'service']):
                    # Extract module information from section
                    lines = section.split('\n')
                    potential_name = ""
                    description_lines = []
                    
                    for line in lines:
                        line = line.strip()
                        if not line:
                            continue
                            
                        # Check if line contains module name
                        if any(indicator in line.lower() for indicator in ['module', 'component', 'system']):
                            if not potential_name or len(line) < len(potential_name):
                                potential_name = self._clean_module_name(line)
                        else:
                            description_lines.append(line)
                    
                    if potential_name and description_lines:
                        description = ' '.join(description_lines[:5])  # First 5 lines
                        if len(description) > 30:
                            status, coverage = self._advanced_status_analysis(description, section)
                            
                            modules.append(ModuleMapping(
                                module_name=potential_name,
                                module_code=self._generate_module_code(potential_name),
                                status=status,
                                coverage_percentage=coverage,
                                description=description[:400],
                                notes="Extracted from section analysis",
                                required=True,
                                available=status in ["Available", "Partial"]
                            ))
        
        return modules
    
    def _parse_context_modules(self, text: str) -> List[ModuleMapping]:
        """Context-aware module parsing using surrounding text analysis"""
        modules = []
        
        # Common module categories with related keywords
        module_categories = {
            "Data Management": ["database", "storage", "data", "repository", "warehouse"],
            "User Interface": ["ui", "interface", "frontend", "dashboard", "portal"],
            "Integration": ["api", "integration", "connector", "interface", "bridge"],
            "Security": ["security", "authentication", "authorization", "encryption", "access"],
            "Analytics": ["analytics", "reporting", "intelligence", "metrics", "dashboard"],
            "Workflow": ["workflow", "process", "automation", "orchestration", "pipeline"],
            "Communication": ["notification", "messaging", "alert", "communication", "email"],
            "Mobile": ["mobile", "app", "responsive", "tablet", "smartphone"]
        }
        
        for category, keywords in module_categories.items():
            # Check if this category is relevant to the document
            keyword_matches = sum(1 for keyword in keywords if keyword.lower() in text.lower())
            
            if keyword_matches >= 2:  # At least 2 keywords found
                # Extract relevant context
                context_sections = []
                sentences = sent_tokenize(text)
                
                for sentence in sentences:
                    if any(keyword in sentence.lower() for keyword in keywords):
                        context_sections.append(sentence)
                
                if context_sections:
                    description = ' '.join(context_sections[:3])  # Combine first 3 relevant sentences
                    status, coverage = self._advanced_status_analysis(description, text)
                    
                    modules.append(ModuleMapping(
                        module_name=f"{category} Module",
                        module_code=self._generate_module_code(category),
                        status=status,
                        coverage_percentage=coverage,
                        description=description[:400],
                        components=', '.join(keywords[:3]),
                        notes="Context-aware extraction",
                        required=True,
                        available=status in ["Available", "Partial"]
                    ))
        
        return modules
    
    def _parse_nlp_modules(self, text: str) -> List[ModuleMapping]:
        """NLP-based module extraction using advanced text analysis"""
        modules = []
        
        try:
            # Use TF-IDF to find important terms
            from sklearn.feature_extraction.text import TfidfVectorizer
            
            sentences = sent_tokenize(text)
            if len(sentences) < 5:
                return modules
            
            # Create TF-IDF vectors
            vectorizer = TfidfVectorizer(
                max_features=50,
                stop_words='english',
                ngram_range=(1, 3),
                min_df=2
            )
            
            tfidf_matrix = vectorizer.fit_transform(sentences)
            feature_names = vectorizer.get_feature_names_out()
            
            # Get top terms
            mean_scores = tfidf_matrix.mean(axis=0).A1
            top_indices = mean_scores.argsort()[-10:][::-1]
            top_terms = [feature_names[i] for i in top_indices]
            
            # Filter terms that could be modules
            module_terms = []
            for term in top_terms:
                if any(indicator in term.lower() for indicator in ['system', 'module', 'service', 'platform', 'engine']):
                    module_terms.append(term)
                elif len(term.split()) > 1 and term.lower() not in ['user interface', 'data management']:
                    # Multi-word technical terms
                    module_terms.append(term)
            
            # Create modules from identified terms
            for term in module_terms[:5]:  # Limit to top 5
                # Find sentences containing this term
                relevant_sentences = [s for s in sentences if term.lower() in s.lower()]
                if relevant_sentences:
                    description = ' '.join(relevant_sentences[:2])
                    status, coverage = self._advanced_status_analysis(description, text)
                    
                    modules.append(ModuleMapping(
                        module_name=term.title(),
                        module_code=self._generate_module_code(term),
                        status=status,
                        coverage_percentage=coverage,
                        description=description[:400],
                        notes="NLP-based extraction using TF-IDF",
                        required=True,
                        available=status in ["Available", "Partial"]
                    ))
        
        except Exception as e:
            print(f"NLP module extraction failed: {e}")
        
        return modules
    
    def _clean_module_name(self, name: str) -> str:
        """Clean and standardize module names"""
        if not name:
            return ""
        
        # Remove common prefixes/suffixes
        name = re.sub(r'^(\d+\.?\s*|[•\-\*]\s*)', '', name.strip())
        name = re.sub(r'\s*(module|component|system|service)\s*$', '', name, flags=re.IGNORECASE)
        
        # Clean special characters but keep meaningful ones
        name = re.sub(r'[^\w\s\-&/()]', '', name)
        
        # Normalize whitespace
        name = ' '.join(name.split())
        
        # Capitalize properly
        if name:
            name = ' '.join(word.capitalize() for word in name.split())
        
        return name.strip()
    
    def _clean_description(self, description: str) -> str:
        """Clean and standardize descriptions"""
        if not description:
            return ""
        
        # Remove excessive whitespace and clean up
        description = ' '.join(description.split())
        
        # Remove common artifacts
        description = re.sub(r'^[-–—:\s]+', '', description)
        description = re.sub(r'[-–—:\s]+$', '', description)
        
        return description.strip()
    
    def _generate_module_code(self, module_name: str) -> str:
        """Generate standardized module codes"""
        if not module_name:
            return "UNKNOWN"
        
        # Take first letter of each significant word
        words = [word for word in module_name.split() if len(word) > 2]
        if len(words) >= 2:
            code = ''.join(word[0].upper() for word in words[:3])
        else:
            code = module_name[:6].upper()
        
        # Ensure minimum length
        if len(code) < 3:
            code += "MOD"
        
        return code[:8]  # Maximum 8 characters
    
    def _advanced_status_analysis(self, description: str, context: str = "") -> Tuple[str, float]:
        """Advanced status analysis using multiple indicators"""
        desc_lower = description.lower()
        context_lower = context.lower()
        combined_text = f"{desc_lower} {context_lower}"
        
        # Strong positive indicators
        strong_positive = [
            "fully implemented", "complete", "production ready", "deployed", 
            "operational", "live", "active", "functional", "working",
            "100%", "fully meets", "exceeds requirements", "proven"
        ]
        
        # Positive indicators
        positive = [
            "implemented", "available", "supports", "provides", "includes", 
            "features", "capable", "ready", "existing", "current"
        ]
        
        # Partial indicators
        partial = [
            "partially", "limited", "basic", "some", "minimal", "developing",
            "in progress", "under development", "customization needed",
            "requires modification", "adaptation required"
        ]
        
        # Negative indicators
        negative = [
            "missing", "lacking", "not available", "unavailable", "planned",
            "future", "to be developed", "not implemented", "gap",
            "requires development", "needs creation"
        ]
        
        # Calculate scores
        strong_pos_score = sum(2 for indicator in strong_positive if indicator in combined_text)
        pos_score = sum(1 for indicator in positive if indicator in combined_text)
        partial_score = sum(1 for indicator in partial if indicator in combined_text)
        neg_score = sum(1 for indicator in negative if indicator in combined_text)
        
        total_positive = strong_pos_score + pos_score
        
        # Determine status and coverage
        if strong_pos_score > 0 or (total_positive > neg_score and total_positive > partial_score):
            if strong_pos_score > 0:
                return "Available", 95.0
            else:
                return "Available", 85.0
        elif partial_score > 0 or (total_positive > 0 and neg_score <= total_positive):
            coverage = max(40.0, min(75.0, 60.0 + (total_positive - neg_score) * 10))
            return "Partial", coverage
        elif neg_score > total_positive:
            return "Missing", 10.0
        else:
            # Default case - assume partial availability
            return "Partial", 65.0
    
    def _advanced_priority_analysis(self, module_name: str, description: str) -> str:
        """Advanced priority analysis"""
        combined_text = f"{module_name} {description}".lower()
        
        # High priority indicators
        high_priority = [
            "critical", "essential", "core", "primary", "key", "vital", 
            "mandatory", "required", "must have", "fundamental", "basic",
            "security", "authentication", "database", "integration"
        ]
        
        # Medium priority indicators
        medium_priority = [
            "important", "significant", "recommended", "should have",
            "useful", "beneficial", "standard", "common"
        ]
        
        high_score = sum(1 for indicator in high_priority if indicator in combined_text)
        medium_score = sum(1 for indicator in medium_priority if indicator in combined_text)
        
        if high_score > medium_score and high_score > 0:
            return "High"
        elif medium_score > 0:
            return "Medium"
        else:
            return "Medium"  # Default
    
    def _extract_components(self, description: str) -> str:
        """Extract component information from description"""
        # Look for component-related keywords
        component_patterns = [
            r"components?[:\s]+([^.]+)",
            r"includes?[:\s]+([^.]+)",
            r"features?[:\s]+([^.]+)",
            r"consists? of[:\s]+([^.]+)"
        ]
        
        for pattern in component_patterns:
            match = re.search(pattern, description, re.IGNORECASE)
            if match:
                return match.group(1).strip()[:200]
        
        return "Components to be defined during detailed analysis"
    
    def _extract_strengths(self, description: str) -> str:
        """Extract strength information from description"""
        strength_keywords = [
            "strength", "advantage", "benefit", "proven", "reliable",
            "robust", "scalable", "efficient", "secure", "flexible"
        ]
        
        sentences = sent_tokenize(description)
        strength_sentences = []
        
        for sentence in sentences:
            if any(keyword in sentence.lower() for keyword in strength_keywords):
                strength_sentences.append(sentence.strip())
        
        if strength_sentences:
            return ' '.join(strength_sentences[:2])[:200]
        
        return "Strengths to be evaluated during detailed assessment"
    
    def _extract_alignment(self, description: str) -> str:
        """Extract alignment information from description"""
        alignment_patterns = [
            r"aligns? with[:\s]+([^.]+)",
            r"meets?[:\s]+([^.]+)",
            r"supports?[:\s]+([^.]+)",
            r"compatible with[:\s]+([^.]+)"
        ]
        
        for pattern in alignment_patterns:
            match = re.search(pattern, description, re.IGNORECASE)
            if match:
                return match.group(1).strip()[:200]
        
        return "Alignment assessment pending detailed requirements review"
    
    def _deduplicate_modules(self, modules: List[ModuleMapping]) -> List[ModuleMapping]:
        """Remove duplicate modules based on similarity"""
        if not modules:
            return modules
        
        unique_modules = []
        seen_names = set()
        
        for module in modules:
            # Create a normalized name for comparison
            normalized_name = re.sub(r'[^\w]', '', module.module_name.lower())
            
            # Check for exact or very similar matches
            is_duplicate = False
            for seen_name in seen_names:
                similarity = self._calculate_string_similarity(normalized_name, seen_name)
                if similarity > 0.8:  # 80% similarity threshold
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                unique_modules.append(module)
                seen_names.add(normalized_name)
        
        return unique_modules
    
    def _calculate_string_similarity(self, str1: str, str2: str) -> float:
        """Calculate similarity between two strings"""
        if not str1 or not str2:
            return 0.0
        
        # Simple Jaccard similarity
        set1 = set(str1.split())
        set2 = set(str2.split())
        
        if not set1 and not set2:
            return 1.0
        
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        
        return intersection / union if union > 0 else 0.0
    
    def _enhance_modules_with_llm_v2(self, modules: List[ModuleMapping], text: str) -> List[ModuleMapping]:
        """Enhanced LLM-based module analysis"""
        if not self.llm_processor or len(modules) == 0:
            return modules
        
        try:
            # Prepare enhanced context for LLM
            module_context = []
            for i, module in enumerate(modules, 1):
                context = f"{i}. {module.module_name}\n   Description: {module.description[:150]}\n   Current Status: {module.status}"
                module_context.append(context)
            
            context_text = '\n'.join(module_context)
            
            prompt = f"""
            As an expert RFP analyst, review these extracted modules and provide enhanced analysis:
            
            {context_text}
            
            For each module, provide:
            1. Refined status (Available/Partial/Missing)
            2. Realistic coverage percentage (0-100%)
            3. Business priority (High/Medium/Low)
            4. Brief improvement suggestion
            
            Format: "Module X: Status=Available|Coverage=85%|Priority=High|Suggestion=Ready for deployment"
            """
            
            llm_response = self.llm_processor._generate_text(prompt, 800)
            
            # Parse LLM response and update modules
            response_lines = [line.strip() for line in llm_response.split('\n') if line.strip()]
            
            for line in response_lines:
                if 'Module' in line and '=' in line:
                    try:
                        # Extract module reference
                        module_ref_match = re.search(r'Module (\d+)', line)
                        if module_ref_match:
                            module_idx = int(module_ref_match.group(1)) - 1
                            
                            if 0 <= module_idx < len(modules):
                                # Parse enhancements
                                if 'Status=' in line:
                                    status_match = re.search(r'Status=([^|]+)', line)
                                    if status_match:
                                        new_status = status_match.group(1).strip()
                                        if new_status in ['Available', 'Partial', 'Missing']:
                                            modules[module_idx].status = new_status
                                
                                if 'Coverage=' in line:
                                    coverage_match = re.search(r'Coverage=(\d+)%', line)
                                    if coverage_match:
                                        coverage = float(coverage_match.group(1))
                                        if 0 <= coverage <= 100:
                                            modules[module_idx].coverage_percentage = coverage
                                
                                if 'Priority=' in line:
                                    priority_match = re.search(r'Priority=([^|]+)', line)
                                    if priority_match:
                                        priority = priority_match.group(1).strip()
                                        if priority in ['High', 'Medium', 'Low']:
                                            modules[module_idx].client_priority = priority
                                
                                if 'Suggestion=' in line:
                                    suggestion_match = re.search(r'Suggestion=(.+)', line)
                                    if suggestion_match:
                                        suggestion = suggestion_match.group(1).strip()
                                        modules[module_idx].notes += f" | LLM Insight: {suggestion}"
                    
                    except (ValueError, IndexError) as e:
                        continue  # Skip invalid lines
        
        except Exception as e:
            print(f"Enhanced LLM module analysis failed: {e}")
        
        return modules
    
    def _create_default_modules(self) -> List[ModuleMapping]:
        """Create comprehensive default modules when parsing fails"""
        return [
            ModuleMapping(
                module_name="Core System Architecture",
                module_code="COREARCH",
                status="Partial",
                coverage_percentage=70.0,
                client_priority="High",
                description="Foundational system architecture and framework components requiring analysis and customization for client-specific requirements.",
                components="Database layer, API framework, security foundation, user management",
                strengths="Proven architecture, scalable design, industry standards compliance",
                alignment="Aligns with enterprise architecture patterns and best practices",
                notes="Default module - requires detailed requirements analysis",
                required=True,
                available=True
            ),
            ModuleMapping(
                module_name="Data Management System",
                module_code="DATAMGMT",
                status="Available",
                coverage_percentage=85.0,
                client_priority="High",
                description="Comprehensive data management capabilities including storage, processing, and governance features.",
                components="Data storage, ETL processes, data quality, backup and recovery",
                strengths="Robust data handling, high performance, compliance ready",
                alignment="Meets standard data management requirements",
                notes="Default module - well-established capabilities",
                required=True,
                available=True
            ),
            ModuleMapping(
                module_name="User Interface Platform",
                module_code="UIPLATF",
                status="Partial",
                coverage_percentage=60.0,
                client_priority="Medium",
                description="User interface and experience platform requiring customization for specific client workflows and branding.",
                components="Web interface, responsive design, accessibility features, theming",
                strengths="Modern UI frameworks, responsive design, accessibility compliance",
                alignment="Requires customization for client-specific user experience",
                notes="Default module - needs UI/UX customization",
                required=True,
                available=True
            )
        ]

    # ========== GAP ANALYSIS METHODS ==========
    
    def parse_gaps_enhanced(self, text: str) -> List[GapItem]:
        """Enhanced gap parsing with advanced pattern recognition"""
        gaps = []
        
        # Strategy 1: Advanced structured gap patterns
        gaps.extend(self._parse_structured_gaps_v2(text))
        
        # Strategy 2: Problem-solution gap analysis
        if len(gaps) < 3:
            gaps.extend(self._parse_problem_solution_gaps(text))
        
        # Strategy 3: Requirement vs capability gaps
        if len(gaps) < 2:
            gaps.extend(self._parse_requirement_capability_gaps(text))
        
        # Strategy 4: Risk-based gap identification
        if len(gaps) < 1:
            gaps.extend(self._parse_risk_based_gaps(text))
        
        # Remove duplicates and enhance with LLM
        gaps = self._deduplicate_gaps(gaps)
        
        if self.llm_processor and gaps:
            gaps = self._enhance_gaps_with_llm_v2(gaps, text)
        
        return gaps if gaps else self._create_default_gaps()
    
    def _parse_structured_gaps_v2(self, text: str) -> List[GapItem]:
        """Advanced structured gap parsing"""
        gaps = []
        
        # Enhanced patterns for gap identification
        gap_patterns = [
            # Pattern 1: Direct gap statements
            r'(?:Gap|Issue|Problem|Challenge|Deficiency)[:\s]+([^.]+?)(?:\.|$)',
            
            # Pattern 2: Missing/lacking statements
            r'(?:Missing|Lacking|Absent|Not available|Unavailable)[:\s]+([^.]+?)(?:\.|$)',
            
            # Pattern 3: Requirements not met
            r'(?:Does not meet|Cannot support|Unable to|Fails to)[:\s]+([^.]+?)(?:\.|$)',
            
            # Pattern 4: Needs/requires statements
            r'(?:Needs|Requires|Must develop|Must implement|Must create)[:\s]+([^.]+?)(?:\.|$)',
            
            # Pattern 5: Limitation statements
            r'(?:Limited|Restricted|Constrained|Insufficient)[:\s]+([^.]+?)(?:\.|$)'
        ]
        
        for pattern in gap_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE)
            
            for match in matches:
                if isinstance(match, str) and len(match.strip()) > 20:
                    gap_description = match.strip()
                    
                    # Classify gap type and severity
                    gap_type = self._classify_gap_type_advanced(gap_description)
                    severity = self._determine_gap_severity_advanced(gap_description, text)
                    
                    gaps.append(GapItem(
                        gap_type=gap_type,
                        description=gap_description[:300],
                        severity=severity,
                        business_impact=self._calculate_business_impact(gap_description, severity),
                        mitigation_strategy=self._generate_mitigation_strategy_advanced(gap_type, gap_description),
                        effort_required=self._estimate_effort_advanced(severity, gap_description),
                        cost_estimate=self._estimate_cost_advanced(severity, gap_description),
                        dependencies=self._identify_dependencies(gap_description),
                        risk_if_unaddressed=self._assess_risk_if_unaddressed(gap_description, severity)
                    ))
        
        return gaps
    
    def _parse_problem_solution_gaps(self, text: str) -> List[GapItem]:
        """Parse gaps from problem-solution analysis"""
        gaps = []
        
        # Look for problem indicators
        problem_keywords = [
            "problem", "issue", "challenge", "concern", "difficulty",
            "obstacle", "barrier", "bottleneck", "weakness", "shortcoming"
        ]
        
        sentences = sent_tokenize(text)
        
        for sentence in sentences:
            sentence_lower = sentence.lower()
            
            # Check if sentence describes a problem
            if any(keyword in sentence_lower for keyword in problem_keywords):
                if len(sentence) > 30:  # Substantial description
                    
                    # Determine if this is a capability gap
                    if any(indicator in sentence_lower for indicator in 
                           ["capability", "feature", "functionality", "ability", "support"]):
                        
                        gap_type = self._classify_gap_type_advanced(sentence)
                        severity = self._determine_gap_severity_advanced(sentence, text)
                        
                        gaps.append(GapItem(
                            gap_type=gap_type,
                            description=f"Problem identified: {sentence[:250]}",
                            severity=severity,
                            business_impact=self._calculate_business_impact(sentence, severity),
                            mitigation_strategy=self._generate_mitigation_strategy_advanced(gap_type, sentence),
                            effort_required=self._estimate_effort_advanced(severity, sentence),
                            cost_estimate=self._estimate_cost_advanced(severity, sentence),
                            risk_if_unaddressed=f"Problem may escalate if not addressed: {severity} impact expected"
                        ))
        
        return gaps[:5]  # Limit to top 5
    
    def _parse_requirement_capability_gaps(self, text: str) -> List[GapItem]:
        """Parse gaps by comparing requirements against capabilities"""
        gaps = []
        
        # Find requirement statements
        requirement_patterns = [
            r'(?:Required|Must have|Mandatory|Essential)[:\s]+([^.]+)',
            r'(?:Client needs|Customer requires|Specification calls for)[:\s]+([^.]+)',
            r'(?:System must|Solution should|Platform needs to)[:\s]+([^.]+)'
        ]
        
        # Find capability statements
        capability_patterns = [
            r'(?:Currently supports|Available|Can provide|Existing)[:\s]+([^.]+)',
            r'(?:Our system|Platform|Solution)[:\s]+([^.]+)',
            r'(?:Implemented|In place|Operational)[:\s]+([^.]+)'
        ]
        
        requirements = []
        capabilities = []
        
        # Extract requirements
        for pattern in requirement_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            requirements.extend([match.strip() for match in matches if len(match.strip()) > 15])
        
        # Extract capabilities
        for pattern in capability_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            capabilities.extend([match.strip() for match in matches if len(match.strip()) > 15])
        
        # Identify gaps by comparing requirements to capabilities
        for requirement in requirements[:5]:  # Limit processing
            # Check if requirement is addressed by any capability
            is_covered = False
            coverage_score = 0
            
            for capability in capabilities:
                similarity = self._calculate_text_similarity(requirement, capability)
                if similarity > 0.3:  # 30% similarity threshold
                    is_covered = True
                    coverage_score = max(coverage_score, similarity)
            
            # If requirement is not well covered, it's a gap
            if not is_covered or coverage_score < 0.6:
                gap_type = self._classify_gap_type_advanced(requirement)
                severity = "High" if not is_covered else "Medium"
                
                gaps.append(GapItem(
                    gap_type=gap_type,
                    description=f"Requirement gap: {requirement[:200]}",
                    severity=severity,
                    business_impact=f"Client requirement may not be fully met (Coverage: {coverage_score*100:.0f}%)",
                    mitigation_strategy=f"Develop capability to address: {requirement[:100]}",
                    effort_required="2-4 weeks" if severity == "Medium" else "4-6 weeks",
                    cost_estimate="$15K-$35K" if severity == "Medium" else "$25K-$50K",
                    risk_if_unaddressed="Client dissatisfaction and potential project failure"
                ))
        
        return gaps
    
    def _parse_risk_based_gaps(self, text: str) -> List[GapItem]:
        """Parse gaps based on risk indicators"""
        gaps = []
        
        risk_indicators = [
            "risk", "concern", "uncertainty", "potential issue", "may impact",
            "could affect", "might cause", "possibility of", "danger of"
        ]
        
        sentences = sent_tokenize(text)
        
        for sentence in sentences:
            if any(indicator in sentence.lower() for indicator in risk_indicators):
                if len(sentence) > 25:
                    gap_type = self._classify_gap_type_advanced(sentence)
                    
                    gaps.append(GapItem(
                        gap_type="Risk",
                        description=f"Risk-based gap: {sentence[:200]}",
                        severity="Medium",
                        business_impact="Potential negative impact on project success",
                        mitigation_strategy="Develop risk mitigation plan and monitoring procedures",
                        effort_required="1-3 weeks",
                        cost_estimate="$10K-$25K",
                        risk_if_unaddressed="Risk may materialize causing project delays or failures"
                    ))
        
        return gaps[:3]  # Limit to top 3
    
    def _classify_gap_type_advanced(self, description: str) -> str:
        """Advanced gap type classification"""
        desc_lower = description.lower()
        
        # Technical gaps
        if any(word in desc_lower for word in 
               ["technical", "technology", "system", "software", "hardware", "infrastructure",
                "architecture", "platform", "framework", "api", "database"]):
            return "Technical"
        
        # Functional gaps
        elif any(word in desc_lower for word in 
                 ["function", "feature", "capability", "workflow", "process", "business logic",
                  "user story", "requirement", "specification"]):
            return "Functional"
        
        # Security gaps
        elif any(word in desc_lower for word in 
                 ["security", "authentication", "authorization", "encryption", "access control",
                  "privacy", "compliance", "audit", "vulnerability"]):
            return "Security"
        
        # Performance gaps
        elif any(word in desc_lower for word in 
                 ["performance", "speed", "latency", "throughput", "scalability", "load",
                  "response time", "efficiency", "optimization"]):
            return "Performance"
        
        # Integration gaps
        elif any(word in desc_lower for word in 
                 ["integration", "interface", "connectivity", "interoperability", "compatibility",
                  "data exchange", "communication", "protocol"]):
            return "Integration"
        
        # Compliance gaps
        elif any(word in desc_lower for word in 
                 ["compliance", "regulation", "standard", "policy", "governance", "regulatory",
                  "legal", "certification", "accreditation"]):
            return "Compliance"
        
        # Operational gaps
        elif any(word in desc_lower for word in 
                 ["operational", "maintenance", "support", "monitoring", "backup", "recovery",
                  "deployment", "configuration", "administration"]):
            return "Operational"
        
        else:
            return "General"
    
    def _determine_gap_severity_advanced(self, description: str, context: str = "") -> str:
        """Advanced gap severity determination"""
        desc_lower = description.lower()
        context_lower = context.lower()
        combined = f"{desc_lower} {context_lower}"
        
        # Critical severity indicators
        critical_indicators = [
            "critical", "severe", "major", "blocking", "showstopper", "fatal",
            "cannot proceed", "project killer", "must have", "essential", "vital"
        ]
        
        # High severity indicators
        high_indicators = [
            "high", "important", "significant", "serious", "substantial", "key",
            "primary", "core", "fundamental", "required", "necessary"
        ]
        
        # Medium severity indicators
        medium_indicators = [
            "medium", "moderate", "intermediate", "standard", "normal", "typical",
            "should have", "recommended", "preferred", "beneficial"
        ]
        
        # Low severity indicators
        low_indicators = [
            "low", "minor", "small", "trivial", "cosmetic", "nice to have",
            "optional", "enhancement", "improvement", "could have"
        ]
        
        # Count indicators
        critical_count = sum(1 for indicator in critical_indicators if indicator in combined)
        high_count = sum(1 for indicator in high_indicators if indicator in combined)
        medium_count = sum(1 for indicator in medium_indicators if indicator in combined)
        low_count = sum(1 for indicator in low_indicators if indicator in combined)
        
        # Determine severity based on counts and context
        if critical_count > 0:
            return "Critical"
        elif high_count > medium_count and high_count > low_count:
            return "High"
        elif low_count > 0 and low_count > high_count:
            return "Low"
        else:
            return "Medium"  # Default
    
    def _calculate_business_impact(self, description: str, severity: str) -> str:
        """Calculate business impact based on description and severity"""
        impact_templates = {
            "Critical": "Severe business impact: Project delivery at risk, potential client relationship damage, significant financial implications",
            "High": "High business impact: Major delays likely, client satisfaction at risk, substantial additional costs expected",
            "Medium": "Moderate business impact: Some delays possible, manageable client communication needed, moderate cost implications",
            "Low": "Low business impact: Minor inconvenience, minimal client communication needed, low cost to address"
        }
        
        base_impact = impact_templates.get(severity, impact_templates["Medium"])
        
        # Add specific context based on description
        desc_lower = description.lower()
        if "security" in desc_lower:
            base_impact += " | Security implications may affect compliance and trust"
        elif "performance" in desc_lower:
            base_impact += " | Performance issues may affect user adoption and satisfaction"
        elif "integration" in desc_lower:
            base_impact += " | Integration challenges may delay go-live and affect functionality"
        
        return base_impact
    
    def _generate_mitigation_strategy_advanced(self, gap_type: str, description: str) -> str:
        """Generate advanced mitigation strategies"""
        base_strategies = {
            "Technical": "Develop technical solution through architecture review, technology evaluation, and implementation planning",
            "Functional": "Design alternative workflow, enhance existing functionality, or develop new features",
            "Security": "Implement additional security controls, conduct security assessment, and establish compliance procedures",
            "Performance": "Optimize system performance through code review, infrastructure enhancement, and load testing",
            "Integration": "Develop integration layer, create API connectors, and establish data synchronization processes",
            "Compliance": "Implement compliance framework, establish audit procedures, and create documentation",
            "Operational": "Develop operational procedures, create monitoring systems, and establish support processes",
            "General": "Conduct detailed analysis, develop action plan, and implement solution with stakeholder approval"
        }
        
        base_strategy = base_strategies.get(gap_type, base_strategies["General"])
        
        # Add specific recommendations based on description content
        desc_lower = description.lower()
        
        if "data" in desc_lower:
            base_strategy += " | Focus on data quality, governance, and migration strategies"
        elif "user" in desc_lower:
            base_strategy += " | Prioritize user experience design and training programs"
        elif "system" in desc_lower:
            base_strategy += " | Emphasize system integration testing and deployment procedures"
        
        return base_strategy
    
    def _estimate_effort_advanced(self, severity: str, description: str) -> str:
        """Advanced effort estimation"""
        base_efforts = {
            "Critical": "6-12 weeks",
            "High": "3-6 weeks",
            "Medium": "2-4 weeks", 
            "Low": "1-2 weeks"
        }
        
        base_effort = base_efforts.get(severity, "2-4 weeks")
        
        # Adjust based on complexity indicators
        desc_lower = description.lower()
        complexity_indicators = ["complex", "multiple", "various", "several", "many", "extensive"]
        
        if any(indicator in desc_lower for indicator in complexity_indicators):
            # Increase effort estimate
            if severity == "Critical":
                base_effort = "8-16 weeks"
            elif severity == "High":
                base_effort = "4-8 weeks"
            elif severity == "Medium":
                base_effort = "3-5 weeks"
        
        return base_effort
    
    def _estimate_cost_advanced(self, severity: str, description: str) -> str:
        """Advanced cost estimation"""
        base_costs = {
            "Critical": "$75K-$150K",
            "High": "$35K-$75K",
            "Medium": "$15K-$35K",
            "Low": "$5K-$15K"
        }
        
        base_cost = base_costs.get(severity, "$15K-$35K")
        
        # Adjust based on resource requirements
        desc_lower = description.lower()
        
        if any(word in desc_lower for word in ["infrastructure", "hardware", "license", "third-party"]):
            # Add infrastructure costs
            if severity in ["Critical", "High"]:
                base_cost += " + Infrastructure costs"
        
        return base_cost
    
    def _identify_dependencies(self, description: str) -> List[str]:
        """Identify dependencies from gap description"""
        dependencies = []
        desc_lower = description.lower()
        
        # Common dependency indicators
        if "integration" in desc_lower:
            dependencies.append("Third-party system availability")
        if "data" in desc_lower:
            dependencies.append("Data source access and quality")
        if "security" in desc_lower:
            dependencies.append("Security team approval and testing")
        if "compliance" in desc_lower:
            dependencies.append("Legal and compliance review")
        if "performance" in desc_lower:
            dependencies.append("Infrastructure capacity and testing environment")
        
        # Default dependencies
        if not dependencies:
            dependencies = ["Stakeholder approval", "Resource allocation", "Timeline coordination"]
        
        return dependencies
    
    def _assess_risk_if_unaddressed(self, description: str, severity: str) -> str:
        """Assess risk if gap is not addressed"""
        risk_templates = {
            "Critical": "Project failure risk: High probability of project cancellation or major scope reduction",
            "High": "Delivery risk: Significant delays and client dissatisfaction likely",
            "Medium": "Quality risk: Reduced functionality and potential rework required",
            "Low": "Minor risk: Small impact on project success and client satisfaction"
        }
        
        base_risk = risk_templates.get(severity, risk_templates["Medium"])
        
        # Add specific risk context
        desc_lower = description.lower()
        if "security" in desc_lower:
            base_risk += " | Security breach potential increases"
        elif "compliance" in desc_lower:
            base_risk += " | Regulatory non-compliance risk"
        elif "performance" in desc_lower:
            base_risk += " | System performance degradation and user complaints"
        
        return base_risk
    
    def _calculate_text_similarity(self, text1: str, text2: str) -> float:
        """Calculate similarity between two text strings"""
        if not text1 or not text2:
            return 0.0
        
        # Convert to lowercase and split into words
        words1 = set(text1.lower().split())
        words2 = set(text2.lower().split())
        
        if not words1 and not words2:
            return 1.0
        
        # Calculate Jaccard similarity
        intersection = len(words1.intersection(words2))
        union = len(words1.union(words2))
        
        return intersection / union if union > 0 else 0.0
    
    def _deduplicate_gaps(self, gaps: List[GapItem]) -> List[GapItem]:
        """Remove duplicate gaps"""
        if not gaps:
            return gaps
        
        unique_gaps = []
        seen_descriptions = []
        
        for gap in gaps:
            # Check for similarity with existing gaps
            is_duplicate = False
            for seen_desc in seen_descriptions:
                similarity = self._calculate_text_similarity(gap.description, seen_desc)
                if similarity > 0.7:  # 70% similarity threshold
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                unique_gaps.append(gap)
                seen_descriptions.append(gap.description)
        
        return unique_gaps
    
    def _enhance_gaps_with_llm_v2(self, gaps: List[GapItem], text: str) -> List[GapItem]:
        """Enhanced LLM-based gap analysis"""
        if not self.llm_processor or not gaps:
            return gaps
        
        try:
            gap_context = []
            for i, gap in enumerate(gaps, 1):
                context = f"{i}. {gap.gap_type} Gap: {gap.description[:100]}\n   Current Severity: {gap.severity}"
                gap_context.append(context)
            
            context_text = '\n'.join(gap_context)
            
            prompt = f"""
            As an expert RFP analyst, review these identified gaps and provide enhanced analysis:
            
            {context_text}
            
            For each gap, provide:
            1. Validated severity (Critical/High/Medium/Low)
            2. Refined gap type if needed
            3. Business priority (Immediate/Short-term/Long-term)
            4. Key mitigation approach
            
            Format: "Gap X: Severity=High|Type=Technical|Priority=Immediate|Approach=Develop custom solution"
            """
            
            llm_response = self.llm_processor._generate_text(prompt, 600)
            
            # Parse and apply enhancements
            response_lines = [line.strip() for line in llm_response.split('\n') if line.strip()]
            
            for line in response_lines:
                if 'Gap' in line and '=' in line:
                    try:
                        gap_ref_match = re.search(r'Gap (\d+)', line)
                        if gap_ref_match:
                            gap_idx = int(gap_ref_match.group(1)) - 1
                            
                            if 0 <= gap_idx < len(gaps):
                                # Apply enhancements
                                if 'Severity=' in line:
                                    severity_match = re.search(r'Severity=([^|]+)', line)
                                    if severity_match:
                                        new_severity = severity_match.group(1).strip()
                                        if new_severity in ['Critical', 'High', 'Medium', 'Low']:
                                            gaps[gap_idx].severity = new_severity
                                
                                if 'Type=' in line:
                                    type_match = re.search(r'Type=([^|]+)', line)
                                    if type_match:
                                        new_type = type_match.group(1).strip()
                                        gaps[gap_idx].gap_type = new_type
                                
                                if 'Approach=' in line:
                                    approach_match = re.search(r'Approach=(.+)', line)
                                    if approach_match:
                                        approach = approach_match.group(1).strip()
                                        gaps[gap_idx].mitigation_strategy = f"LLM Enhanced: {approach}"
                    
                    except (ValueError, IndexError):
                        continue
        
        except Exception as e:
            print(f"Enhanced LLM gap analysis failed: {e}")
        
        return gaps
    
    def _create_default_gaps(self) -> List[GapItem]:
        """Create default gaps when parsing fails"""
        return [
            GapItem(
                gap_type="Analysis",
                description="Comprehensive gap analysis required - detailed review needed to identify specific capability gaps against client requirements",
                severity="Medium",
                business_impact="Analysis phase needed to determine actual gaps and their business impact",
                mitigation_strategy="Conduct detailed requirements workshop with client stakeholders and technical team",
                effort_required="2-3 weeks",
                cost_estimate="$15K-$25K",
                dependencies=["Client availability", "Requirements documentation", "Technical team allocation"],
                risk_if_unaddressed="May miss critical gaps leading to project scope creep and client dissatisfaction"
            ),
            GapItem(
                gap_type="Documentation",
                description="Complete technical documentation and specification gaps need to be addressed for comprehensive understanding",
                severity="Low",
                business_impact="Documentation gaps may slow development and increase communication overhead",
                mitigation_strategy="Create comprehensive technical documentation and specification review process",
                effort_required="1-2 weeks",
                cost_estimate="$8K-$15K",
                dependencies=["Technical team input", "Client specification review"],
                risk_if_unaddressed="Development delays and increased rework due to unclear requirements"
            )
        ]

    # ========== WIN/LOSS ANALYSIS METHODS ==========
    
    def parse_win_loss_enhanced(self, text: str) -> List[WinLossFactor]:
        """Enhanced win/loss factor parsing with comprehensive analysis"""
        factors = []
        
        # Strategy 1: Advanced structured win/loss patterns
        factors.extend(self._parse_structured_win_loss_v2(text))
        
        # Strategy 2: Historical performance analysis
        if len(factors) < 3:
            factors.extend(self._parse_historical_performance(text))
        
        # Strategy 3: Competitive advantage/disadvantage analysis
        if len(factors) < 2:
            factors.extend(self._parse_competitive_analysis(text))
        
        # Strategy 4: Client feedback and testimonial analysis
        if len(factors) < 1:
            factors.extend(self._parse_client_feedback_analysis(text))
        
        # Enhance with LLM and remove duplicates
        factors = self._deduplicate_win_loss_factors(factors)
        
        if self.llm_processor and factors:
            factors = self._enhance_win_loss_with_llm_v2(factors, text)
        
        return factors if factors else self._create_default_win_loss_factors()
    
    def _parse_structured_win_loss_v2(self, text: str) -> List[WinLossFactor]:
        """Advanced structured win/loss factor parsing"""
        factors = []
        
        # Enhanced win patterns
        win_patterns = [
            r'(?:Win|Success|Victory|Achievement|Strength|Advantage)[:\s]+(.*?)(?=(?:Loss|Win|Success|$))',
            r'(?:Successful|Won|Achieved|Delivered|Exceeded)[:\s]+(.*?)(?=(?:Failed|Lost|$))',
            r'(?:Key strength|Competitive advantage|Success factor)[:\s]+(.*?)(?=(?:Weakness|Challenge|$))',
            r'(?:Why we won|Success story|Positive outcome)[:\s]+(.*?)(?:\n\n|$)',
            r'(?:Client praised|Recognized for|Commended)[:\s]+(.*?)(?:\n|$)'
        ]
        
        # Enhanced loss patterns
        loss_patterns = [
            r'(?:Loss|Failure|Defeat|Challenge|Weakness|Disadvantage)[:\s]+(.*?)(?=(?:Win|Success|$))',
            r'(?:Failed|Lost|Missed|Delayed|Cancelled)[:\s]+(.*?)(?=(?:Won|Succeeded|$))',
            r'(?:Key weakness|Competitive disadvantage|Failure factor)[:\s]+(.*?)(?=(?:Strength|Success|$))',
            r'(?:Why we lost|Failure story|Negative outcome)[:\s]+(.*?)(?:\n\n|$)',
            r'(?:Client complained|Criticized for|Issues with)[:\s]+(.*?)(?:\n|$)'
        ]
        
        # Process win patterns
        for pattern in win_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE | re.DOTALL)
            for match in matches:
                if isinstance(match, str) and len(match.strip()) > 25:
                    description = match.strip()[:300]
                    
                    factors.append(WinLossFactor(
                        factor_type="Win",
                        description=description,
                        impact=self._determine_factor_impact_advanced(description),
                        confidence=self._calculate_confidence_score(description, text),
                        category=self._categorize_factor_advanced(description)
                    ))
        
        # Process loss patterns
        for pattern in loss_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE | re.DOTALL)
            for match in matches:
                if isinstance(match, str) and len(match.strip()) > 25:
                    description = match.strip()[:300]
                    
                    factors.append(WinLossFactor(
                        factor_type="Loss",
                        description=description,
                        impact=self._determine_factor_impact_advanced(description),
                        confidence=self._calculate_confidence_score(description, text),
                        category=self._categorize_factor_advanced(description)
                    ))
        
        return factors
    
    def _parse_historical_performance(self, text: str) -> List[WinLossFactor]:
        """Parse historical performance indicators"""
        factors = []
        
        # Performance indicators
        performance_keywords = {
            "positive": ["exceeded", "surpassed", "outperformed", "delivered ahead", "under budget", 
                        "client satisfaction", "repeat business", "referral", "testimonial"],
            "negative": ["over budget", "delayed", "missed deadline", "client dissatisfaction", 
                        "scope creep", "quality issues", "performance problems", "cancelled"]
        }
        
        sentences = sent_tokenize(text)
        
        for sentence in sentences:
            sentence_lower = sentence.lower()
            
            # Check for positive performance indicators
            positive_matches = sum(1 for keyword in performance_keywords["positive"] if keyword in sentence_lower)
            negative_matches = sum(1 for keyword in performance_keywords["negative"] if keyword in sentence_lower)
            
            if positive_matches > 0 and len(sentence) > 30:
                factors.append(WinLossFactor(
                    factor_type="Win",
                    description=f"Historical success: {sentence[:250]}",
                    impact=self._determine_factor_impact_advanced(sentence),
                    confidence=0.8 + (positive_matches * 0.05),  # Higher confidence with more indicators
                    category=self._categorize_factor_advanced(sentence)
                ))
            
            elif negative_matches > 0 and len(sentence) > 30:
                factors.append(WinLossFactor(
                    factor_type="Loss",
                    description=f"Historical challenge: {sentence[:250]}",
                    impact=self._determine_factor_impact_advanced(sentence),
                    confidence=0.8 + (negative_matches * 0.05),
                    category=self._categorize_factor_advanced(sentence)
                ))
        
        return factors[:8]  # Limit to top 8
    
    def _parse_competitive_analysis(self, text: str) -> List[WinLossFactor]:
        """Parse competitive advantages and disadvantages"""
        factors = []
        
        competitive_patterns = [
            # Advantage patterns
            (r'(?:competitive advantage|market leader|industry leader|best in class)[:\s]+(.*?)(?:\n|$)', "Win"),
            (r'(?:outperform|superior to|better than|ahead of)[:\s]+(.*?)(?:\n|$)', "Win"),
            (r'(?:unique capability|proprietary|exclusive|patented)[:\s]+(.*?)(?:\n|$)', "Win"),
            
            # Disadvantage patterns
            (r'(?:competitive disadvantage|behind competitors|lagging)[:\s]+(.*?)(?:\n|$)', "Loss"),
            (r'(?:competitors offer|others provide|market expects)[:\s]+(.*?)(?:\n|$)', "Loss"),
            (r'(?:catching up|need to match|industry standard)[:\s]+(.*?)(?:\n|$)', "Loss")
        ]
        
        for pattern, factor_type in competitive_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE)
            for match in matches:
                if len(match.strip()) > 20:
                    factors.append(WinLossFactor(
                        factor_type=factor_type,
                        description=f"Competitive analysis: {match.strip()[:250]}",
                        impact=self._determine_factor_impact_advanced(match),
                        confidence=0.85,
                        category="Competitive"
                    ))
        
        return factors
    
    def _parse_client_feedback_analysis(self, text: str) -> List[WinLossFactor]:
        """Parse client feedback and testimonials"""
        factors = []
        
        # Client feedback patterns
        feedback_patterns = [
            # Positive feedback
            (r'(?:client said|customer feedback|testimonial)[:\s]+["\']([^"\']+)["\']', "Win"),
            (r'(?:praised for|commended|recognized|awarded)[:\s]+(.*?)(?:\n|$)', "Win"),
            (r'(?:client satisfaction|positive feedback|happy with)[:\s]+(.*?)(?:\n|$)', "Win"),
            
            # Negative feedback
            (r'(?:client complained|customer concerns|feedback issues)[:\s]+(.*?)(?:\n|$)', "Loss"),
            (r'(?:criticized for|concerns about|problems with)[:\s]+(.*?)(?:\n|$)', "Loss"),
            (r'(?:client dissatisfaction|negative feedback|unhappy)[:\s]+(.*?)(?:\n|$)', "Loss")
        ]
        
        for pattern, factor_type in feedback_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE)
            for match in matches:
                if len(match.strip()) > 15:
                    factors.append(WinLossFactor(
                        factor_type=factor_type,
                        description=f"Client feedback: {match.strip()[:250]}",
                        impact=self._determine_factor_impact_advanced(match),
                        confidence=0.9,  # High confidence for direct client feedback
                        category="Client Relationship"
                    ))
        
        return factors
    
    def _determine_factor_impact_advanced(self, description: str) -> str:
        """Advanced impact determination"""
        desc_lower = description.lower()
        
        # High impact indicators
        high_impact = [
            "critical", "major", "significant", "substantial", "dramatic", "transformative",
            "game changer", "breakthrough", "revolutionary", "market leading", "industry first"
        ]
        
        # Medium impact indicators
        medium_impact = [
            "important", "notable", "considerable", "meaningful", "valuable", "beneficial",
            "improvement", "enhancement", "advantage", "positive", "good"
        ]
        
        # Low impact indicators
        low_impact = [
            "minor", "small", "slight", "modest", "limited", "marginal", "incremental",
            "basic", "standard", "routine", "normal", "typical"
        ]
        
        high_count = sum(1 for indicator in high_impact if indicator in desc_lower)
        medium_count = sum(1 for indicator in medium_impact if indicator in desc_lower)
        low_count = sum(1 for indicator in low_impact if indicator in desc_lower)
        
        if high_count > 0:
            return "High"
        elif low_count > medium_count and low_count > 0:
            return "Low"
        else:
            return "Medium"
    
    def _calculate_confidence_score(self, description: str, context: str) -> float:
        """Calculate confidence score for win/loss factor"""
        base_confidence = 0.7
        
        # Increase confidence for specific indicators
        desc_lower = description.lower()
        
        # Quantitative data increases confidence
        if any(indicator in desc_lower for indicator in 
               ["percent", "%", "million", "thousand", "dollars", "$", "days", "weeks", "months"]):
            base_confidence += 0.1
        
        # Specific examples increase confidence
        if any(indicator in desc_lower for indicator in 
               ["project", "client", "implementation", "specific", "example", "case study"]):
            base_confidence += 0.05
        
        # Recent timeframes increase confidence
        if any(indicator in desc_lower for indicator in 
               ["recent", "last year", "2024", "2023", "this year", "currently"]):
            base_confidence += 0.08
        
        # Cap at 0.95
        return min(0.95, base_confidence)
    
    def _categorize_factor_advanced(self, description: str) -> str:
        """Advanced factor categorization"""
        desc_lower = description.lower()
        
        # Technical category
        if any(word in desc_lower for word in 
               ["technical", "technology", "system", "platform", "architecture", "software", 
                "development", "coding", "integration", "api", "database"]):
            return "Technical"
        
        # Team/Resource category
        elif any(word in desc_lower for word in 
                 ["team", "staff", "resource", "skill", "expertise", "experience", "talent",
                  "developer", "engineer", "consultant", "project manager"]):
            return "Team"
        
        # Financial category
        elif any(word in desc_lower for word in 
                 ["cost", "budget", "price", "financial", "money", "dollar", "investment",
                  "roi", "savings", "expensive", "cheap"]):
            return "Financial"
        
        # Timeline category
        elif any(word in desc_lower for word in 
                 ["time", "schedule", "deadline", "delivery", "fast", "quick", "slow",
                  "delayed", "ahead", "behind", "timeline"]):
            return "Timeline"
        
        # Client relationship category
        elif any(word in desc_lower for word in 
                 ["client", "customer", "stakeholder", "relationship", "communication",
                  "feedback", "satisfaction", "trust", "partnership"]):
            return "Client"
        
        # Quality category
        elif any(word in desc_lower for word in 
                 ["quality", "reliability", "performance", "stability", "robust",
                  "bug", "error", "testing", "qa"]):
            return "Quality"
        
        # Process category
        elif any(word in desc_lower for word in 
                 ["process", "methodology", "approach", "workflow", "procedure",
                  "agile", "waterfall", "management"]):
            return "Process"
        
        # Market/Business category
        elif any(word in desc_lower for word in 
                 ["market", "business", "strategy", "competitive", "industry",
                  "opportunity", "growth", "expansion"]):
            return "Business"
        
        else:
            return "General"
    
    def _deduplicate_win_loss_factors(self, factors: List[WinLossFactor]) -> List[WinLossFactor]:
        """Remove duplicate win/loss factors"""
        if not factors:
            return factors
        
        unique_factors = []
        seen_descriptions = []
        
        for factor in factors:
            # Check for similarity with existing factors
            is_duplicate = False
            for seen_desc in seen_descriptions:
                similarity = self._calculate_text_similarity(factor.description, seen_desc)
                if similarity > 0.75:  # 75% similarity threshold
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                unique_factors.append(factor)
                seen_descriptions.append(factor.description)
        
        return unique_factors
    
    def _enhance_win_loss_with_llm_v2(self, factors: List[WinLossFactor], text: str) -> List[WinLossFactor]:
        """Enhanced LLM-based win/loss factor analysis"""
        if not self.llm_processor or not factors:
            return factors
        
        try:
            factor_context = []
            for i, factor in enumerate(factors, 1):
                context = f"{i}. {factor.factor_type}: {factor.description[:120]}\n   Category: {factor.category}, Impact: {factor.impact}"
                factor_context.append(context)
            
            context_text = '\n'.join(factor_context[:10])  # Limit to first 10 factors
            
            prompt = f"""
            As an expert RFP analyst, review these win/loss factors and provide strategic insights:
            
            {context_text}
            
            For each factor, provide:
            1. Strategic importance (Critical/High/Medium/Low)
            2. Actionability (Immediate/Short-term/Long-term/Monitor)
            3. Market relevance (High/Medium/Low)
            4. Brief strategic recommendation
            
            Format: "Factor X: Importance=High|Action=Immediate|Market=High|Rec=Leverage in proposals"
            """
            
            llm_response = self.llm_processor._generate_text(prompt, 700)
            
            # Parse and apply enhancements
            response_lines = [line.strip() for line in llm_response.split('\n') if line.strip()]
            
            for line in response_lines:
                if 'Factor' in line and '=' in line:
                    try:
                        factor_ref_match = re.search(r'Factor (\d+)', line)
                        if factor_ref_match:
                            factor_idx = int(factor_ref_match.group(1)) - 1
                            
                            if 0 <= factor_idx < len(factors):
                                # Extract strategic insights
                                if 'Importance=' in line:
                                    importance_match = re.search(r'Importance=([^|]+)', line)
                                    if importance_match:
                                        importance = importance_match.group(1).strip()
                                        # Map to impact levels
                                        if importance in ['Critical', 'High']:
                                            factors[factor_idx].impact = "High"
                                        elif importance == 'Medium':
                                            factors[factor_idx].impact = "Medium"
                                        else:
                                            factors[factor_idx].impact = "Low"
                                
                                if 'Rec=' in line:
                                    rec_match = re.search(r'Rec=(.+)', line)
                                    if rec_match:
                                        recommendation = rec_match.group(1).strip()
                                        # Add recommendation to description
                                        original_desc = factors[factor_idx].description
                                        factors[factor_idx].description = f"{original_desc} | Strategic insight: {recommendation}"
                    
                    except (ValueError, IndexError):
                        continue
        
        except Exception as e:
            print(f"Enhanced LLM win/loss analysis failed: {e}")
        
        return factors
    
    def _create_default_win_loss_factors(self) -> List[WinLossFactor]:
        """Create default win/loss factors when parsing fails"""
        return [
            WinLossFactor(
                factor_type="Win",
                description="Proven technical expertise and successful project delivery track record in similar implementations",
                impact="High",
                confidence=0.85,
                category="Technical"
            ),
            WinLossFactor(
                factor_type="Win",
                description="Strong client relationships and high satisfaction scores from previous engagements",
                impact="High",
                confidence=0.80,
                category="Client"
            ),
            WinLossFactor(
                factor_type="Win",
                description="Competitive pricing model with fixed-cost approach reducing client financial risk",
                impact="Medium",
                confidence=0.75,
                category="Financial"
            ),
            WinLossFactor(
                factor_type="Loss",
                description="Limited historical data available - requires analysis of specific win/loss patterns for this market segment",
                impact="Medium",
                confidence=0.60,
                category="General"
            ),
            WinLossFactor(
                factor_type="Unknown",
                description="Comprehensive win/loss analysis needed to identify specific success factors and areas for improvement",
                impact="Medium",
                confidence=0.50,
                category="Analysis"
            )
        ]

    # ========== REQUIREMENTS ANALYSIS METHODS ==========
    
    def parse_requirements_enhanced(self, text: str) -> List[Requirement]:
        """Enhanced requirements parsing with comprehensive analysis"""
        requirements = []
        
        # Strategy 1: Advanced structured requirement patterns
        requirements.extend(self._parse_structured_requirements_v2(text))
        
        # Strategy 2: MoSCoW method parsing (Must/Should/Could/Won't)
        if len(requirements) < 3:
            requirements.extend(self._parse_moscow_requirements(text))
        
        # Strategy 3: User story and use case parsing
        if len(requirements) < 2:
            requirements.extend(self._parse_user_stories(text))
        
        # Strategy 4: Compliance and regulatory requirements
        if len(requirements) < 1:
            requirements.extend(self._parse_compliance_requirements(text))
        
        # Remove duplicates and enhance
        requirements = self._deduplicate_requirements(requirements)
        
        if self.llm_processor and requirements:
            requirements = self._enhance_requirements_with_llm(requirements, text)
        
        return requirements if requirements else self._create_default_requirements()
    
    def _parse_structured_requirements_v2(self, text: str) -> List[Requirement]:
        """Advanced structured requirements parsing"""
        requirements = []
        
        # Enhanced requirement patterns
        requirement_patterns = [
            # Functional requirements
            (r'(?:Functional requirement|FR-?\d*)[:\s]+(.*?)(?=(?:Non-functional|Technical|Performance|$))', "Functional"),
            (r'(?:The system shall|The solution must|The platform should)[:\s]+(.*?)(?:\.|$)', "Functional"),
            (r'(?:User must be able to|System must support|Application shall)[:\s]+(.*?)(?:\.|$)', "Functional"),
            
            # Non-functional requirements
            (r'(?:Non-functional requirement|NFR-?\d*|Performance requirement)[:\s]+(.*?)(?=(?:Functional|Technical|$))', "Performance"),
            (r'(?:Performance|Scalability|Availability|Reliability)[:\s]+(.*?)(?:\.|$)', "Performance"),
            (r'(?:System must handle|Response time|Throughput)[:\s]+(.*?)(?:\.|$)', "Performance"),
            
            # Technical requirements
            (r'(?:Technical requirement|TR-?\d*)[:\s]+(.*?)(?=(?:Functional|Performance|$))', "Technical"),
            (r'(?:Technology stack|Platform|Infrastructure)[:\s]+(.*?)(?:\.|$)', "Technical"),
            (r'(?:Integration with|API|Database)[:\s]+(.*?)(?:\.|$)', "Integration"),
            
            # Security requirements
            (r'(?:Security requirement|Authentication|Authorization|Encryption)[:\s]+(.*?)(?:\.|$)', "Security"),
            (r'(?:Access control|Data protection|Privacy)[:\s]+(.*?)(?:\.|$)', "Security"),
            
            # Compliance requirements
            (r'(?:Compliance|Regulatory|Standards?)[:\s]+(.*?)(?:\.|$)', "Compliance"),
            (r'(?:Must comply with|According to|As per)[:\s]+(.*?)(?:\.|$)', "Compliance")
        ]
        
        for pattern, req_type in requirement_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE | re.DOTALL)
            
            for match in matches:
                if isinstance(match, str) and len(match.strip()) > 20:
                    description = match.strip()[:400]
                    
                    # Determine priority and capability
                    priority = self._determine_requirement_priority_advanced(description)
                    capability = self._assess_our_capability_advanced(description, text)
                    effort = self._calculate_requirement_effort(capability, req_type)
                    
                    requirements.append(Requirement(
                        requirement_type=req_type,
                        description=description,
                        priority=priority,
                        our_capability=capability,
                        effort_to_meet=effort
                    ))
        
        return requirements
    
    def _parse_moscow_requirements(self, text: str) -> List[Requirement]:
        """Parse requirements using MoSCoW method"""
        requirements = []
        
        moscow_patterns = [
            (r'(?:Must have|Mandatory|Required|Essential|Critical)[:\s]+(.*?)(?=(?:Should have|Could have|Won\'t have|$))', "Must Have"),
            (r'(?:Should have|Important|Recommended|Preferred)[:\s]+(.*?)(?=(?:Must have|Could have|Won\'t have|$))', "Should Have"),
            (r'(?:Could have|Nice to have|Optional|If time permits)[:\s]+(.*?)(?=(?:Must have|Should have|Won\'t have|$))', "Could Have"),
            (r'(?:Won\'t have|Out of scope|Future|Next phase)[:\s]+(.*?)(?=(?:Must have|Should have|Could have|$))', "Won't Have")
        ]
        
        for pattern, priority in moscow_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE | re.DOTALL)
            
            for match in matches:
                if len(match.strip()) > 25:
                    description = match.strip()[:400]
                    req_type = self._classify_requirement_type_advanced(description)
                    capability = self._assess_our_capability_advanced(description, text)
                    
                    if priority != "Won't Have":  # Skip out-of-scope requirements
                        requirements.append(Requirement(
                            requirement_type=req_type,
                            description=description,
                            priority=priority,
                            our_capability=capability,
                            effort_to_meet=self._calculate_requirement_effort(capability, req_type)
                        ))
        
        return requirements
    
    def _parse_user_stories(self, text: str) -> List[Requirement]:
        """Parse user stories and use cases"""
        requirements = []
        
        # User story patterns
        user_story_patterns = [
            r'(?:As a|As an)\s+([^,]+),\s*I want\s+([^,]+),?\s*(?:so that|in order to)?\s*(.*?)(?:\.|$)',
            r'(?:User story|Story)[:\s]*([^.]+?)(?:\.|$)',
            r'(?:Use case|UC-?\d*)[:\s]+(.*?)(?=(?:Use case|User story|$))'
        ]
        
        for pattern in user_story_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE)
            
            for match in matches:
                if isinstance(match, tuple):
                    # Full user story format
                    if len(match) == 3:
                        user_role, action, benefit = match
                        description = f"As a {user_role}, I want {action} so that {benefit}"
                    else:
                        description = ' '.join(match)
                else:
                    description = match
                
                if len(description.strip()) > 30:
                    req_type = self._classify_requirement_type_advanced(description)
                    priority = self._determine_requirement_priority_advanced(description)
                    capability = self._assess_our_capability_advanced(description, text)
                    
                    requirements.append(Requirement(
                        requirement_type="Functional",  # User stories are typically functional
                        description=description[:400],
                        priority=priority,
                        our_capability=capability,
                        effort_to_meet=self._calculate_requirement_effort(capability, "Functional")
                    ))
        
        return requirements[:10]  # Limit to top 10 user stories
    
    def _parse_compliance_requirements(self, text: str) -> List[Requirement]:
        """Parse compliance and regulatory requirements"""
        requirements = []
        
        # Compliance indicators
        compliance_keywords = [
            "GDPR", "HIPAA", "SOX", "PCI DSS", "ISO 27001", "SOC 2",
            "compliance", "regulatory", "audit", "certification",
            "data protection", "privacy", "security standards"
        ]
        
        sentences = sent_tokenize(text)
        
        for sentence in sentences:
            sentence_lower = sentence.lower()
            
            # Check if sentence mentions compliance
            compliance_matches = sum(1 for keyword in compliance_keywords if keyword.lower() in sentence_lower)
            
            if compliance_matches > 0 and len(sentence) > 30:
                priority = "Must Have" if any(keyword in sentence_lower for keyword in 
                                            ["mandatory", "required", "must", "shall"]) else "Should Have"
                
                capability = self._assess_our_capability_advanced(sentence, text)
                
                requirements.append(Requirement(
                    requirement_type="Compliance",
                    description=f"Compliance requirement: {sentence[:350]}",
                    priority=priority,
                    our_capability=capability,
                    effort_to_meet=self._calculate_requirement_effort(capability, "Compliance")
                ))
        
        return requirements[:5]  # Limit to top 5 compliance requirements
    
    def _classify_requirement_type_advanced(self, description: str) -> str:
        """Advanced requirement type classification"""
        desc_lower = description.lower()
        
        # Security requirements
        if any(word in desc_lower for word in 
               ["security", "authentication", "authorization", "encryption", "access control",
                "privacy", "confidential", "secure", "login", "password"]):
            return "Security"
        
        # Performance requirements
        elif any(word in desc_lower for word in 
                 ["performance", "speed", "response time", "latency", "throughput", "load",
                  "scalability", "concurrent", "users", "transactions"]):
            return "Performance"
        
        # Integration requirements
        elif any(word in desc_lower for word in 
                 ["integration", "interface", "api", "connect", "sync", "import", "export",
                  "third party", "external", "web service"]):
            return "Integration"
        
        # Data requirements
        elif any(word in desc_lower for word in 
                 ["data", "database", "storage", "backup", "recovery", "migration",
                  "format", "structure", "schema"]):
            return "Data"
        
        # UI/UX requirements
        elif any(word in desc_lower for word in 
                 ["user interface", "ui", "ux", "usability", "accessibility", "responsive",
                  "mobile", "design", "layout", "navigation"]):
            return "UI/UX"
        
        # Compliance requirements
        elif any(word in desc_lower for word in 
                 ["compliance", "regulatory", "audit", "standard", "certification",
                  "gdpr", "hipaa", "sox", "pci"]):
            return "Compliance"
        
        # Functional requirements (default)
        else:
            return "Functional"
    
    def _determine_requirement_priority_advanced(self, description: str) -> str:
        """Advanced requirement priority determination"""
        desc_lower = description.lower()
        
        # Must have indicators
        must_have = [
            "must", "mandatory", "required", "essential", "critical", "vital",
            "shall", "needed", "necessary", "cannot work without", "blocker"
        ]
        
        # Should have indicators
        should_have = [
            "should", "important", "significant", "recommended", "preferred",
            "expected", "standard", "typical", "normal practice"
        ]
        
        # Could have indicators
        could_have = [
            "could", "nice to have", "optional", "if possible", "desirable",
            "enhancement", "improvement", "future", "phase 2"
        ]
        
        must_count = sum(1 for indicator in must_have if indicator in desc_lower)
        should_count = sum(1 for indicator in should_have if indicator in desc_lower)
        could_count = sum(1 for indicator in could_have if indicator in desc_lower)
        
        if must_count > 0:
            return "Must Have"
        elif could_count > should_count and could_count > 0:
            return "Could Have"
        elif should_count > 0:
            return "Should Have"
        else:
            # Default based on requirement type keywords
            if any(word in desc_lower for word in ["security", "compliance", "data"]):
                return "Must Have"
            else:
                return "Should Have"
    
    def _assess_our_capability_advanced(self, description: str, context: str = "") -> str:
        """Advanced capability assessment"""
        desc_lower = description.lower()
        context_lower = context.lower()
        combined = f"{desc_lower} {context_lower}"
        
        # Full capability indicators
        full_indicators = [
            "fully support", "complete", "available", "implemented", "existing",
            "current", "in place", "operational", "ready", "proven"
        ]
        
        # Partial capability indicators
        partial_indicators = [
            "partially", "limited", "basic", "some", "customization needed",
            "modification required", "enhancement needed", "adaptation"
        ]
        
        # No capability indicators
        none_indicators = [
            "missing", "not available", "not supported", "requires development",
            "needs to be built", "custom development", "new feature"
        ]
        
        full_count = sum(1 for indicator in full_indicators if indicator in combined)
        partial_count = sum(1 for indicator in partial_indicators if indicator in combined)
        none_count = sum(1 for indicator in none_indicators if indicator in combined)
        
        if full_count > partial_count and full_count > none_count:
            return "Full"
        elif none_count > partial_count and none_count > 0:
            return "None"
        else:
            return "Partial"
    
    def _calculate_requirement_effort(self, capability: str, req_type: str) -> str:
        """Calculate effort required based on capability and requirement type"""
        base_efforts = {
            ("Full", "Functional"): "None",
            ("Full", "Technical"): "1 week",
            ("Full", "Performance"): "1-2 weeks",
            ("Full", "Security"): "1-2 weeks",
            ("Full", "Integration"): "2-3 weeks",
            ("Full", "Compliance"): "2-4 weeks",
            ("Full", "Data"): "1-2 weeks",
            ("Full", "UI/UX"): "1-3 weeks",
            
            ("Partial", "Functional"): "2-4 weeks",
            ("Partial", "Technical"): "3-5 weeks",
            ("Partial", "Performance"): "4-6 weeks",
            ("Partial", "Security"): "4-8 weeks",
            ("Partial", "Integration"): "4-8 weeks",
            ("Partial", "Compliance"): "6-10 weeks",
            ("Partial", "Data"): "3-6 weeks",
            ("Partial", "UI/UX"): "4-8 weeks",
            
            ("None", "Functional"): "6-12 weeks",
            ("None", "Technical"): "8-16 weeks",
            ("None", "Performance"): "10-20 weeks",
            ("None", "Security"): "12-24 weeks",
            ("None", "Integration"): "8-16 weeks",
            ("None", "Compliance"): "16-32 weeks",
            ("None", "Data"): "8-16 weeks",
            ("None", "UI/UX"): "10-20 weeks"
        }
        
        return base_efforts.get((capability, req_type), "4-8 weeks")
    
    def _deduplicate_requirements(self, requirements: List[Requirement]) -> List[Requirement]:
        """Remove duplicate requirements"""
        if not requirements:
            return requirements
        
        unique_requirements = []
        seen_descriptions = []
        
        for req in requirements:
            # Check for similarity with existing requirements
            is_duplicate = False
            for seen_desc in seen_descriptions:
                similarity = self._calculate_text_similarity(req.description, seen_desc)
                if similarity > 0.7:  # 70% similarity threshold
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                unique_requirements.append(req)
                seen_descriptions.append(req.description)
        
        return unique_requirements
    
    def _enhance_requirements_with_llm(self, requirements: List[Requirement], text: str) -> List[Requirement]:
        """Enhanced LLM-based requirements analysis"""
        if not self.llm_processor or not requirements:
            return requirements
        
        try:
            req_context = []
            for i, req in enumerate(requirements, 1):
                context = f"{i}. {req.requirement_type}: {req.description[:100]}\n   Priority: {req.priority}, Capability: {req.our_capability}"
                req_context.append(context)
            
            context_text = '\n'.join(req_context[:8])  # Limit to first 8 requirements
            
            prompt = f"""
            As an expert RFP analyst, review these requirements and provide enhanced analysis:
            
            {context_text}
            
            For each requirement, provide:
            1. Business criticality (Critical/High/Medium/Low)
            2. Implementation complexity (Complex/Moderate/Simple)
            3. Risk level (High/Medium/Low)
            4. Brief implementation note
            
            Format: "Req X: Criticality=High|Complexity=Moderate|Risk=Low|Note=Standard feature"
            """
            
            llm_response = self.llm_processor._generate_text(prompt, 600)
            
            # Parse and apply enhancements
            response_lines = [line.strip() for line in llm_response.split('\n') if line.strip()]
            
            for line in response_lines:
                if 'Req' in line and '=' in line:
                    try:
                        req_ref_match = re.search(r'Req (\d+)', line)
                        if req_ref_match:
                            req_idx = int(req_ref_match.group(1)) - 1
                            
                            if 0 <= req_idx < len(requirements):
                                # Apply enhancements
                                if 'Criticality=' in line:
                                    criticality_match = re.search(r'Criticality=([^|]+)', line)
                                    if criticality_match:
                                        criticality = criticality_match.group(1).strip()
                                        # Map to priority
                                        if criticality in ['Critical', 'High']:
                                            requirements[req_idx].priority = "Must Have"
                                        elif criticality == 'Medium':
                                            requirements[req_idx].priority = "Should Have"
                                        else:
                                            requirements[req_idx].priority = "Could Have"
                                
                                if 'Complexity=' in line:
                                    complexity_match = re.search(r'Complexity=([^|]+)', line)
                                    if complexity_match:
                                        complexity = complexity_match.group(1).strip()
                                        # Adjust effort based on complexity
                                        current_effort = requirements[req_idx].effort_to_meet
                                        if complexity == 'Complex' and 'weeks' in current_effort:
                                            # Increase effort by 50%
                                            requirements[req_idx].effort_to_meet = f"Extended: {current_effort}"
                                
                                if 'Note=' in line:
                                    note_match = re.search(r'Note=(.+)', line)
                                    if note_match:
                                        note = note_match.group(1).strip()
                                        # Add note to description
                                        original_desc = requirements[req_idx].description
                                        requirements[req_idx].description = f"{original_desc} | Analysis: {note}"
                    
                    except (ValueError, IndexError):
                        continue
        
        except Exception as e:
            print(f"Enhanced LLM requirements analysis failed: {e}")
        
        return requirements
    
    def _create_default_requirements(self) -> List[Requirement]:
        """Create default requirements when parsing fails"""
        return [
            Requirement(
                requirement_type="Functional",
                description="Core system functionality as specified in client requirements document - comprehensive analysis needed to define specific functional capabilities",
                priority="Must Have",
                our_capability="Partial",
                effort_to_meet="4-8 weeks"
            ),
            Requirement(
                requirement_type="Performance",
                description="System performance requirements including response times, throughput, and scalability to handle expected user load and data volumes",
                priority="Must Have",
                our_capability="Partial",
                effort_to_meet="3-6 weeks"
            ),
            Requirement(
                requirement_type="Security",
                description="Security and access control requirements including authentication, authorization, data encryption, and compliance with industry standards",
                priority="Must Have",
                our_capability="Full",
                effort_to_meet="2-4 weeks"
            ),
            Requirement(
                requirement_type="Integration",
                description="Integration requirements with existing client systems, third-party services, and data sources as specified in the RFP",
                priority="Should Have",
                our_capability="Partial",
                effort_to_meet="4-8 weeks"
            ),
            Requirement(
                requirement_type="UI/UX",
                description="User interface and user experience requirements including accessibility, responsiveness, and usability standards",
                priority="Should Have",
                our_capability="Partial",
                effort_to_meet="6-10 weeks"
            )
        ]

In [176]:
class RFPFeasibilityAnalyzer:
    """Main analyzer that determines if we can respond to the RFP"""
    
    def __init__(self):
        self.processor = DocumentProcessor()
        self.modules = []
        self.gaps = []
        self.win_loss_factors = []
        self.requirements = []
        self.document_summaries = {}
        
    def _parse_llm_text_output(self, text: str, doc_type: str) -> Dict[str, Any]:
        """Parse plain text output from LLM into summary, confidence, and key points"""
        try:
            # Initialize default values
            result = {
                'llm_analysis': f"Summary of {doc_type}: [Text not available]",
                'confidence': 0.85,
                'key_points': [
                    f"Key insight 1 for {doc_type}",
                    f"Key insight 2 for {doc_type}",
                    f"Key insight 3 for {doc_type}"
                ]
            }
            
            # Extract summary
            summary_match = re.search(r'Summary: (.*?)(?=\nConfidence:|\Z)', text, re.DOTALL)
            if summary_match:
                result['llm_analysis'] = summary_match.group(1).strip()
            
            # Extract confidence
            confidence_match = re.search(r'Confidence: (\d*\.?\d*)', text)
            if confidence_match:
                try:
                    confidence = float(confidence_match.group(1))
                    if 0 <= confidence <= 1:
                        result['confidence'] = confidence
                except ValueError:
                    pass
            
            # Extract key points
            key_points_match = re.search(r'Key Points:\n((?:- [^\n]*\n?)+)', text, re.DOTALL)
            if key_points_match:
                points = key_points_match.group(1).split('\n- ')
                points = [p.strip('- ').strip() for p in points if p.strip('- ').strip()]
                if len(points) >= 2:
                    result['key_points'] = points[:3]
            
            return result
        except Exception as e:
            print(f"Error parsing LLM text output for {doc_type}: {e}")
            return result
    
    def load_and_analyze_documents(self, file_paths: Dict[str, str]) -> Dict[str, Any]:
        """Load documents and extract structured information"""
        results = {
            'documents_processed': 0,
            'total_content_length': 0,
            'extraction_summary': {}
        }
        
        for doc_type, file_path in file_paths.items():
            print(f"\n📄 Analyzing {doc_type}...")
            text = self.processor.extract_text_from_pdf(file_path)
            
            if not text:
                print(f"  ⚠️  Warning: Could not extract text from {file_path}")
                continue
            
            print(f"  ✓ Extracted {len(text)} characters")
            results['documents_processed'] += 1
            results['total_content_length'] += len(text)
            
            # Store document summary
            self.document_summaries[doc_type] = {
                'length': len(text),
                'preview': text[:500],
                'sentences': len(sent_tokenize(text)),
                'llm_summary': '',
                'confidence': 0.0,
                'key_points': []
            }
            
            # Get LLM analysis for the document
            if self.processor.llm_processor:
                print(f"  🤖 Running AI analysis...")
                prompt = f"""
                You are an expert RFP analyst. Analyze the following document of type '{doc_type}' and provide:
                - A summary of the document (max 200 words).
                - A confidence score (0.0 to 1.0) for the accuracy of the analysis.
                - Key points (list 2-3 bullet points summarizing critical insights).
                Document text: {text[:1500]}
                Return in the format:
                Summary: [summary]
                Confidence: [score]
                Key Points:
                - [point 1]
                - [point 2]
                - [point 3]
                """
                llm_result = self.processor.llm_processor._generate_text(prompt, max_length=1000)
                parsed_result = self._parse_llm_text_output(llm_result, doc_type)
                print(f"  ✓ AI insights generated")
                self.document_summaries[doc_type]['llm_summary'] = parsed_result['llm_analysis']
                self.document_summaries[doc_type]['confidence'] = parsed_result['confidence']
                self.document_summaries[doc_type]['key_points'] = parsed_result['key_points']
            
            # Extract information based on document type
            if doc_type == 'module_matching':
                self.modules = self.processor.parse_modules_enhanced(text)
                results['extraction_summary']['modules'] = len(self.modules)
                print(f"  ✓ Extracted {len(self.modules)} modules")
                
            elif doc_type == 'gap_analysis':
                self.gaps = self.processor.parse_gaps_enhanced(text)
                results['extraction_summary']['gaps'] = len(self.gaps)
                print(f"  ✓ Extracted {len(self.gaps)} gaps")
                
            elif doc_type == 'win_loss':
                self.win_loss_factors = self.processor.parse_win_loss_enhanced(text)
                results['extraction_summary']['win_loss_factors'] = len(self.win_loss_factors)
                print(f"  ✓ Extracted {len(self.win_loss_factors)} win/loss factors")
                
            elif doc_type == 'customer_needs':
                self.requirements = self.processor.parse_requirements_enhanced(text)
                results['extraction_summary']['requirements'] = len(self.requirements)
                print(f"  ✓ Extracted {len(self.requirements)} requirements")
        
        return results
    
    def calculate_feasibility(self) -> RFPFeasibilityAssessment:
        """Calculate overall feasibility of responding to RFP with LLM insights"""
        
        # Module analysis
        total_modules = len(self.modules)
        available_modules = len([m for m in self.modules if m.status in ["Available", "Partial"]])
        partial_modules = len([m for m in self.modules if m.status == "Partial"])
        missing_modules = len([m for m in self.modules if m.status in ["Missing", "Unknown"]])
        
        module_coverage = (available_modules / total_modules * 100) if total_modules > 0 else 0
        
        # Gap analysis
        critical_gaps = [g for g in self.gaps if g.severity in ['Critical', 'High']]
        medium_gaps = [g for g in self.gaps if g.severity == 'Medium']
        low_gaps = [g for g in self.gaps if g.severity == 'Low']
        
        # Requirements analysis
        must_have_reqs = [r for r in self.requirements if r.priority == 'Must Have']
        met_must_haves = [r for r in must_have_reqs if r.our_capability in ['Full', 'Partial']]
        requirement_coverage = (len(met_must_haves) / len(must_have_reqs) * 100) if must_have_reqs else 100
        
        # Win/Loss analysis
        win_factors = [f for f in self.win_loss_factors if f.factor_type == 'Win']
        loss_factors = [f for f in self.win_loss_factors if f.factor_type == 'Loss']
        win_loss_ratio = len(win_factors) / (len(loss_factors) + 1)  # +1 to avoid division by zero
        
        # Calculate confidence score (0-100)
        confidence_score = 0
        
        # Module coverage (30% weight)
        confidence_score += module_coverage * 0.3
        
        # Requirement coverage (30% weight)
        confidence_score += requirement_coverage * 0.3
        
        # Gap severity (20% weight - inverse)
        gap_penalty = len(critical_gaps) * 10 + len(medium_gaps) * 5 + len(low_gaps) * 2
        confidence_score += max(0, 20 - gap_penalty)
        
        # Win/Loss history (20% weight)
        if win_loss_ratio > 2:
            confidence_score += 20
        elif win_loss_ratio > 1:
            confidence_score += 15
        elif win_loss_ratio > 0.5:
            confidence_score += 10
        else:
            confidence_score += 5
        
        # Determine if we can respond
        can_respond = confidence_score >= 60 and len(critical_gaps) <= 2
        
        # Calculate win probability (includes market factors)
        win_probability = confidence_score * 0.8 if can_respond else confidence_score * 0.5
        
        # Get LLM insights
        print("\n🤖 Generating AI insights...")
        module_insights = self.processor.llm_processor.analyze_module_coverage(self.modules)
        gap_insights = self.processor.llm_processor.analyze_gaps_impact(self.gaps)
        win_loss_insights = self.processor.llm_processor.analyze_win_loss_patterns(self.win_loss_factors)
        
        # Generate SWOT analysis
        strengths = [
            f"Strong win/loss ratio: {win_loss_ratio:.1f}" if win_loss_ratio > 1 else None,
            f"{available_modules} modules available" if available_modules > 0 else None,
            f"{len(met_must_haves)} must-have requirements met" if met_must_haves else None,
        ] + [f.description for f in win_factors[:2]]
        if "strength" in module_insights.lower():
            strengths.append(f"AI Insight: {module_insights[:100]}...")
        strengths = [s for s in strengths if s]
        
        weaknesses = [
            f"{missing_modules} modules missing" if missing_modules > 0 else None,
            f"{len(critical_gaps)} critical gaps" if critical_gaps else None,
            f"Low requirement coverage: {requirement_coverage:.0f}%" if requirement_coverage < 70 else None,
        ] + [f.description for f in loss_factors[:2]]
        if "weakness" in gap_insights.lower() or "concern" in gap_insights.lower():
            weaknesses.append(f"AI Insight: {gap_insights[:100]}...")
        weaknesses = [w for w in weaknesses if w]
        
        opportunities = [
            "Leverage partial modules" if partial_modules > 0 else None,
            "Use past win experience" if win_factors else None,
            "Mitigate identified gaps" if self.gaps else None,
        ]
        if "opportunity" in win_loss_insights.lower():
            opportunities.append(f"AI Insight: {win_loss_insights[:100]}...")
        opportunities = [o for o in opportunities if o]
        
        threats = [
            "Critical gaps may disqualify" if critical_gaps else None,
            "Loss factors remain relevant" if loss_factors else None,
            "High development effort needed" if missing_modules > total_modules * 0.3 else None,
        ]
        threats = [t for t in threats if t]
        
        # Generate required actions
        required_actions = []
        for gap in critical_gaps[:3]:
            required_actions.append((
                f"Address {gap.gap_type} gap: {gap.description[:50]}...",
                gap.severity,
                gap.effort_required,
                "Gap Team"
            ))
        unmet_must_haves = [r for r in must_have_reqs if r.our_capability in ['None', 'Partial']]
        for req in unmet_must_haves[:2]:
            required_actions.append((
                f"Develop {req.requirement_type}: {req.description[:40]}...",
                "High",
                req.effort_to_meet,
                "Development Team"
            ))
        critical_missing_modules = [m for m in self.modules if m.status == "Missing" and m.client_priority == "High"]
        for module in critical_missing_modules[:2]:
            required_actions.append((
                f"Develop module: {module.module_name}",
                "Critical",
                "2-4 weeks",
                "Technical Team"
            ))
        
        # Risk assessment
        risks = []
        if len(critical_gaps) > 0:
            risks.append(RiskAssessment(
                risk_type="Technical",
                description=f"{len(critical_gaps)} critical gaps may prevent delivery",
                probability="High" if len(critical_gaps) > 2 else "Medium",
                impact="High",
                mitigation_plan="Fast-track gap closure"
            ))
        if module_coverage < 70:
            risks.append(RiskAssessment(
                risk_type="Delivery",
                description=f"Low module coverage ({module_coverage:.0f}%) risks timeline",
                probability="High",
                impact="Medium",
                mitigation_plan="Partner for missing modules"
            ))
        if requirement_coverage < 80:
            risks.append(RiskAssessment(
                risk_type="Commercial",
                description="Requirement gaps may impact satisfaction",
                probability="Medium",
                impact="High",
                mitigation_plan="Communicate roadmap to client"
            ))
        
        # Investment and timeline estimates
        investment_parts = []
        timeline_parts = []
        if missing_modules > 0:
            investment_parts.append(f"${missing_modules * 50}K-${missing_modules * 100}K for modules")
            timeline_parts.append(f"{missing_modules * 2}-{missing_modules * 4} weeks for modules")
        if len(critical_gaps) > 0:
            investment_parts.append(f"${len(critical_gaps) * 25}K-${len(critical_gaps) * 50}K for gaps")
            timeline_parts.append(f"{len(critical_gaps)}-{len(critical_gaps) * 2} weeks for gaps")
        investment_required = " + ".join(investment_parts) if investment_parts else "Minimal investment"
        timeline_estimate = ", ".join(timeline_parts) if timeline_parts else "Immediate start"
        
        # Resource requirements
        resource_needs = []
        if missing_modules > 5:
            resource_needs.append(f"{missing_modules // 2} developers")
        if len(critical_gaps) > 3:
            resource_needs.append(f"{len(critical_gaps) // 2} architects")
        if len(unmet_must_haves) > 0:
            resource_needs.append("1-2 business analysts")
        resource_requirements = ", ".join(resource_needs) if resource_needs else "Existing team sufficient"
        
        # Create initial assessment
        initial_assessment = RFPFeasibilityAssessment(
            can_respond=can_respond,
            confidence_score=confidence_score,
            win_probability=win_probability,
            total_modules=total_modules,
            available_modules=available_modules,
            partial_modules=partial_modules,
            missing_modules=missing_modules,
            critical_gaps=critical_gaps,
            all_gaps=self.gaps,
            strengths=strengths,
            weaknesses=weaknesses,
            opportunities=opportunities,
            threats=threats,
            required_actions=required_actions,
            risks=risks,
            investment_required=investment_required,
            timeline_estimate=timeline_estimate,
            resource_requirements=resource_requirements
        )
        
        # Get LLM-enhanced insights
        executive_recommendation = self.processor.llm_processor.generate_executive_recommendation(initial_assessment)
        competitive_analysis = self.processor.llm_processor.analyze_competitive_positioning(
            self.modules, self.gaps, self.win_loss_factors
        )
        action_plan = self.processor.llm_processor.generate_action_plan(
            self.gaps, [m for m in self.modules if m.status == "Missing"],
            timeline_estimate, investment_required
        )
        
        # Generate detailed rationale
        detailed_rationale = f"""
## Feasibility Analysis Details

### Module Readiness
- Total Modules: {total_modules}
- Available: {available_modules} ({module_coverage:.1f}%)
- Partial: {partial_modules} ({partial_modules/total_modules*100:.1f}%)
- Missing: {missing_modules} ({missing_modules/total_modules*100:.1f}%)
**AI Analysis:** {module_insights}

### Requirements Coverage
- Must-Have Requirements: {len(must_have_reqs)}
- Met: {len(met_must_haves)} ({requirement_coverage:.1f}%)
- Gaps: {len(unmet_must_haves)} requirements

### Gap Assessment
- Critical Gaps: {len(critical_gaps)}
- Risks: {len(risks)}
- Risk Area: {"Technical" if len(critical_gaps) > 2 else "Delivery" if module_coverage < 70 else "Commercial"}
**AI Analysis:** {gap_insights}

### Historical Performance
- Win/Loss Ratio: {win_loss_ratio:.2f}
- Success Factors: {len(win_factors)}
- Challenges: {len(loss_factors)}
**AI Analysis:** {win_loss_insights}

### Investment & Timeline
- Investment: {investment_required}
- Timeline: {timeline_estimate}
- Resources: {resource_requirements}

### AI Action Plan:
{action_plan}
        """.strip()
        
        # Competitive positioning
        competitive_positioning = f"""
## Competitive Analysis

{competitive_analysis}

### Summary:
- Market Position: {"Strong" if win_loss_ratio > 1.5 else "Average" if win_loss_ratio > 0.7 else "Weak"}
- Differentiators: {', '.join([s[:30] + '...' for s in strengths[:3]]) if strengths else 'Limited differentiators'}
- Gaps: {', '.join([w[:30] + '...' for w in weaknesses[:3]]) if weaknesses else 'No major gaps'}
        """.strip()
        
        # Executive summary
        executive_summary = f"""
## RFP Response Feasibility Assessment

**Decision: {"GO - Proceed" if can_respond else "NO-GO - Do Not Proceed"}**

### Key Metrics:
- Confidence: {confidence_score:.1f}%
- Win Probability: {win_probability:.1f}%
- Module Coverage: {module_coverage:.1f}%
- Requirement Coverage: {requirement_coverage:.1f}%

### Summary:
Analyzed {len(self.document_summaries)} documents ({sum(d['length'] for d in self.document_summaries.values())} characters):
- Modules: {available_modules}/{total_modules} ({module_coverage:.0f}%)
- Requirements: {len(met_must_haves)}/{len(must_have_reqs)} ({requirement_coverage:.0f}%)
- Gaps: {len(critical_gaps)} critical, {len(medium_gaps)} medium, {len(low_gaps)} low
- Win/Loss: Ratio {win_loss_ratio:.1f} ({len(self.win_loss_factors)} factors)

### AI Recommendation:
{executive_recommendation}

### Next Steps:
1. {required_actions[0][0] if required_actions else "Proceed with RFP response"}
2. {required_actions[1][0] if len(required_actions) > 1 else "Allocate resources"}
3. {required_actions[2][0] if len(required_actions) > 2 else "Schedule review"}
        """.strip()
        
        # Update assessment
        initial_assessment.executive_summary = executive_summary
        initial_assessment.detailed_rationale = detailed_rationale
        initial_assessment.competitive_positioning = competitive_positioning
        return initial_assessment

In [177]:
class RFPFeasibilityReportGenerator:
    """Generates comprehensive RFP feasibility report"""
    
    def __init__(self):
        self.styles = getSampleStyleSheet()
        self.title_style = ParagraphStyle(
            'CustomTitle',
            parent=self.styles['Heading1'],
            fontSize=24,
            spaceAfter=30,
            alignment=TA_CENTER,
            textColor=colors.HexColor('#1e3a8a')
        )
        self.heading_style = ParagraphStyle(
            'CustomHeading',
            parent=self.styles['Heading2'],
            fontSize=16,
            spaceAfter=12,
            textColor=colors.HexColor('#1e3a8a')
        )
        self.subheading_style = ParagraphStyle(
            'CustomSubheading',
            parent=self.styles['Heading3'],
            fontSize=13,
            spaceAfter=8,
            textColor=colors.HexColor('#1e3a8a')
        )
        
    def create_go_no_go_decision(self, assessment: RFPFeasibilityAssessment) -> List:
        """Create GO/NO-GO decision section"""
        elements = []
        
        elements.append(Paragraph("GO / NO-GO Decision", self.title_style))
        elements.append(Spacer(1, 20))
        
        # Decision box
        decision_color = colors.green if assessment.can_respond else colors.red
        decision_text = "GO - Proceed with RFP Response" if assessment.can_respond else "NO-GO - Do Not Proceed"
        
        decision_data = [[Paragraph(decision_text, ParagraphStyle(
            'Decision',
            parent=self.styles['Heading1'],
            fontSize=20,
            alignment=TA_CENTER,
            textColor=colors.white
        ))]]
        
        decision_table = Table(decision_data, colWidths=[6*inch])
        decision_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, -1), decision_color),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('TOPPADDING', (0, 0), (-1, -1), 20),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 20),
            ('LEFTPADDING', (0, 0), (-1, -1), 10),
            ('RIGHTPADDING', (0, 0), (-1, -1), 10),
        ]))
        
        elements.append(decision_table)
        elements.append(Spacer(1, 20))
        
        # Key metrics dashboard
        metrics_data = [
            ['Metric', 'Value', 'Status'],
            ['Confidence Score', f"{assessment.confidence_score:.1f}%", 
             '✅' if assessment.confidence_score >= 70 else '⚠️' if assessment.confidence_score >= 50 else '❌'],
            ['Win Probability', f"{assessment.win_probability:.1f}%",
             '✅' if assessment.win_probability >= 60 else '⚠️' if assessment.win_probability >= 40 else '❌'],
            ['Module Coverage', f"{assessment.available_modules}/{assessment.total_modules} ({(assessment.available_modules/assessment.total_modules*100) if assessment.total_modules > 0 else 0:.0f}%)",
             '✅' if assessment.total_modules > 0 and assessment.available_modules/assessment.total_modules >= 0.7 else '⚠️' if assessment.total_modules > 0 and assessment.available_modules/assessment.total_modules >= 0.5 else '❌'],
            ['Critical Gaps', str(len(assessment.critical_gaps)),
             '✅' if len(assessment.critical_gaps) == 0 else '⚠️' if len(assessment.critical_gaps) <= 2 else '❌'],
            ['Investment Required', assessment.investment_required.split(' + ')[0] if ' + ' in assessment.investment_required else assessment.investment_required[:30] + '...',
             '✅' if 'Minimal' in assessment.investment_required else '⚠️'],
        ]
        
        metrics_table = Table(metrics_data, colWidths=[2*inch, 3*inch, 1*inch])
        metrics_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ('ALIGN', (2, 0), (2, -1), 'CENTER'),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('TOPPADDING', (0, 0), (-1, -1), 8),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f0f0f0')])
        ]))
        
        elements.append(metrics_table)
        elements.append(Spacer(1, 30))
        
        return elements
    
    def create_executive_summary(self, assessment: RFPFeasibilityAssessment) -> List:
        """Create executive summary with proper formatting"""
        elements = []
        
        elements.append(Paragraph("Executive Summary", self.heading_style))
        elements.append(Spacer(1, 12))
        
        # Convert markdown-style formatting to ReportLab
        summary_parts = assessment.executive_summary.split('\n')
        for part in summary_parts:
            if part.startswith('##'):
                # Main heading
                clean_text = part.replace('##', '').strip()
                elements.append(Paragraph(clean_text, self.subheading_style))
            elif part.startswith('**') and part.endswith('**'):
                # Bold text
                clean_text = part.replace('**', '')
                elements.append(Paragraph(clean_text, ParagraphStyle(
                    'BoldText',
                    parent=self.styles['Normal'],
                    fontName='Helvetica-Bold',
                    fontSize=11
                )))
            elif part.startswith('###'):
                # Subheading
                clean_text = part.replace('###', '').strip()
                elements.append(Paragraph(clean_text, self.subheading_style))
            elif part.startswith('-'):
                # Bullet point
                clean_text = '• ' + part[1:].strip()
                elements.append(Paragraph(clean_text, self.styles['Normal']))
            elif part.strip():
                # Regular paragraph
                elements.append(Paragraph(part.strip(), self.styles['Normal']))
            
            if part.strip():
                elements.append(Spacer(1, 6))
        
        elements.append(Spacer(1, 20))
        return elements
    
    def create_module_analysis_section(self, modules: List[ModuleMapping], assessment: RFPFeasibilityAssessment) -> List:
        """Create detailed module analysis section"""
        elements = []
        
        elements.append(Paragraph("Module Analysis", self.heading_style))
        elements.append(Spacer(1, 12))
        
        # Module summary
        summary_text = f"""Total modules analyzed: {assessment.total_modules} | 
        Available: {assessment.available_modules} | 
        Partial: {assessment.partial_modules} | 
        Missing: {assessment.missing_modules}"""
        
        elements.append(Paragraph(summary_text, self.styles['Normal']))
        elements.append(Spacer(1, 12))
        
        if modules:
            # Create module table
            module_data = [['Module Code', 'Module Name', 'Status', 'Priority', 'Coverage', 'Notes']]
            
            # Sort modules by status and priority
            sorted_modules = sorted(modules, key=lambda x: (
                0 if x.status == "Missing" else 1 if x.status == "Partial" else 2,
                0 if x.client_priority == "High" else 1 if x.client_priority == "Medium" else 2
            ))
            
            for module in sorted_modules[:20]:  # Limit to top 20 for space
                status_color = colors.green if module.status == "Available" else colors.orange if module.status == "Partial" else colors.red if module.status == "Missing" else colors.grey
                
                # Ensure module name is complete and readable
                module_name = module.module_name
                if len(module_name) > 40:
                    module_name = module_name[:37] + "..."
                
                # Clean up notes
                notes = module.notes
                if len(notes) > 60:
                    notes = notes[:57] + "..."
                
                module_data.append([
                    module.module_code,
                    Paragraph(module_name, self.styles['Normal']),
                    Paragraph(module.status, ParagraphStyle('Status', parent=self.styles['Normal'], textColor=status_color, fontSize=8)),
                    module.client_priority,
                    f"{module.coverage_percentage:.0f}%",
                    Paragraph(notes, ParagraphStyle('Notes', parent=self.styles['Normal'], fontSize=8))
                ])
            
            module_table = Table(module_data, colWidths=[0.8*inch, 1.5*inch, 0.8*inch, 0.7*inch, 0.7*inch, 2*inch])
            module_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, -1), 8),
                ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f5f5f5')])
            ]))
            
            elements.append(module_table)
            
            if len(modules) > 20:
                elements.append(Spacer(1, 6))
                elements.append(Paragraph(f"Note: Showing 20 of {len(modules)} modules. See appendix for complete list.", 
                                        ParagraphStyle('Note', parent=self.styles['Normal'], fontSize=8, textColor=colors.grey)))
        
        elements.append(Spacer(1, 20))
        return elements
    
    def create_gap_analysis_section(self, assessment: RFPFeasibilityAssessment) -> List:
        """Create enhanced gap analysis section"""
        elements = []
        
        elements.append(Paragraph("Gap Analysis", self.heading_style))
        elements.append(Spacer(1, 12))
        
        # Gap summary by severity
        gap_summary = f"""Critical Gaps: {len([g for g in assessment.all_gaps if g.severity == 'Critical'])} | 
        High: {len([g for g in assessment.all_gaps if g.severity == 'High'])} | 
        Medium: {len([g for g in assessment.all_gaps if g.severity == 'Medium'])} | 
        Low: {len([g for g in assessment.all_gaps if g.severity == 'Low'])}"""
        
        elements.append(Paragraph(gap_summary, self.styles['Normal']))
        elements.append(Spacer(1, 12))
        
        if assessment.all_gaps:
            gap_data = [['Type', 'Description', 'Severity', 'Impact', 'Mitigation', 'Effort']]
            
            # Sort by severity
            severity_order = {'Critical': 0, 'High': 1, 'Medium': 2, 'Low': 3}
            sorted_gaps = sorted(assessment.all_gaps, key=lambda x: severity_order.get(x.severity, 4))
            
            for gap in sorted_gaps[:15]:  # Top 15 gaps
                severity_color = colors.red if gap.severity == 'Critical' else colors.orange if gap.severity == 'High' else colors.yellow if gap.severity == 'Medium' else colors.green
                
                gap_data.append([
                    gap.gap_type,
                    Paragraph(gap.description[:60] + '...' if len(gap.description) > 60 else gap.description, self.styles['Normal']),
                    Paragraph(gap.severity, ParagraphStyle('Severity', parent=self.styles['Normal'], textColor=severity_color, fontName='Helvetica-Bold')),
                    Paragraph(gap.business_impact[:40] + '...' if len(gap.business_impact) > 40 else gap.business_impact, self.styles['Normal']),
                    Paragraph(gap.mitigation_strategy[:50] + '...' if len(gap.mitigation_strategy) > 50 else gap.mitigation_strategy, self.styles['Normal']),
                    gap.effort_required
                ])
            
            gap_table = Table(gap_data, colWidths=[0.8*inch, 1.8*inch, 0.7*inch, 1.2*inch, 1.5*inch, 0.7*inch])
            gap_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, -1), 8),
                ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f5f5f5')])
            ]))
            
            elements.append(gap_table)
        else:
            elements.append(Paragraph("No gaps identified in the analysis.", self.styles['Normal']))
        
        elements.append(Spacer(1, 20))
        return elements

    def create_swot_analysis(self, assessment: RFPFeasibilityAssessment) -> List:
        """Create SWOT analysis section with size constraints"""
        elements = []
        
        elements.append(Paragraph("SWOT Analysis", self.heading_style))
        elements.append(Spacer(1, 12))
        
        swot_style = ParagraphStyle(
            'SWOT',
            parent=self.styles['Normal'],
            fontSize=9,
            leading=12,
            spaceAfter=4
        )
        
        # Helper function to format SWOT items
        def format_items(items, title, max_items=4, max_length=100):
            content = [Paragraph(f"<b>{title}</b>", swot_style), Spacer(1, 6)]
            for i, item in enumerate(items[:max_items], 1):
                if item:
                    text = item[:max_length] + ('...' if len(item) > max_length else '')
                    content.append(Paragraph(f"{i}. {text}", swot_style))
                    content.append(Spacer(1, 4))
            return content
        
        # Create SWOT content
        strengths_content = format_items(assessment.strengths, "STRENGTHS")
        weaknesses_content = format_items(assessment.weaknesses, "WEAKNESSES")
        opportunities_content = format_items(assessment.opportunities, "OPPORTUNITIES")
        threats_content = format_items(assessment.threats, "THREATS")
        
        # Create 2x2 SWOT grid
        swot_data = [
            [strengths_content, weaknesses_content],
            [opportunities_content, threats_content]
        ]
        
        swot_table = Table(swot_data, colWidths=[3*inch, 3*inch])
        swot_table.setStyle(TableStyle([
            ('GRID', (0, 0), (-1, -1), 2, colors.HexColor('#1e3a8a')),
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('BACKGROUND', (0, 0), (0, 0), colors.HexColor('#e8f5e9')),  # Green for strengths
            ('BACKGROUND', (1, 0), (1, 0), colors.HexColor('#ffebee')),  # Red for weaknesses
            ('BACKGROUND', (0, 1), (0, 1), colors.HexColor('#e3f2fd')),  # Blue for opportunities
            ('BACKGROUND', (1, 1), (1, 1), colors.HexColor('#fff3e0')),  # Orange for threats
            ('TOPPADDING', (0, 0), (-1, -1), 12),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 12),
            ('LEFTPADDING', (0, 0), (-1, -1), 12),
            ('RIGHTPADDING', (0, 0), (-1, -1), 12),
        ]))
        
        elements.append(swot_table)
        if any(len(items) > 4 for items in [assessment.strengths, assessment.weaknesses, assessment.opportunities, assessment.threats]):
            elements.append(Spacer(1, 6))
            elements.append(Paragraph("Note: Additional SWOT items available in appendix.", 
                                     ParagraphStyle('Note', parent=self.styles['Normal'], fontSize=8, textColor=colors.grey)))
        
        elements.append(Spacer(1, 20))
        return elements
        
    def create_action_plan(self, assessment: RFPFeasibilityAssessment) -> List:
        """Create detailed action plan"""
        elements = []
        
        elements.append(Paragraph("Action Plan", self.heading_style))
        elements.append(Spacer(1, 12))
        
        if assessment.required_actions:
            action_data = [['#', 'Action', 'Priority', 'Timeline', 'Owner']]
            
            for i, (action, priority, timeline, owner) in enumerate(assessment.required_actions[:10], 1):
                priority_color = colors.red if priority == 'Critical' else colors.orange if priority == 'High' else colors.black
                
                action_data.append([
                    str(i),
                    Paragraph(action, self.styles['Normal']),
                    Paragraph(priority, ParagraphStyle('Priority', parent=self.styles['Normal'], 
                                                       textColor=priority_color, fontName='Helvetica-Bold')),
                    timeline,
                    owner
                ])
            
            action_table = Table(action_data, colWidths=[0.3*inch, 3.2*inch, 0.8*inch, 1*inch, 1.2*inch])
            action_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, -1), 9),
                ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f0f0f0')])
            ]))
            
            elements.append(action_table)
        else:
            elements.append(Paragraph("No immediate actions required.", self.styles['Normal']))
        
        elements.append(Spacer(1, 20))
        return elements
    
    def create_risk_assessment_section(self, assessment: RFPFeasibilityAssessment) -> List:
        """Create risk assessment section"""
        elements = []
        
        elements.append(Paragraph("Risk Assessment", self.heading_style))
        elements.append(Spacer(1, 12))
        
        if assessment.risks:
            risk_data = [['Risk Type', 'Description', 'Probability', 'Impact', 'Mitigation']]
            
            for risk in assessment.risks:
                prob_color = colors.red if risk.probability == 'High' else colors.orange if risk.probability == 'Medium' else colors.green
                impact_color = colors.red if risk.impact == 'High' else colors.orange if risk.impact == 'Medium' else colors.green
                
                risk_data.append([
                    risk.risk_type,
                    Paragraph(risk.description[:80] + '...' if len(risk.description) > 80 else risk.description, self.styles['Normal']),
                    Paragraph(risk.probability, ParagraphStyle('Risk', parent=self.styles['Normal'], textColor=prob_color)),
                    Paragraph(risk.impact, ParagraphStyle('Risk', parent=self.styles['Normal'], textColor=impact_color)),
                    Paragraph(risk.mitigation_plan[:60] + '...' if len(risk.mitigation_plan) > 60 else risk.mitigation_plan, self.styles['Normal'])
                ])
            
            risk_table = Table(risk_data, colWidths=[0.8*inch, 2.3*inch, 0.8*inch, 0.8*inch, 1.8*inch])
            risk_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, -1), 9),
                ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f0f0f0')])
            ]))
            
            elements.append(risk_table)
        else:
            elements.append(Paragraph("No significant risks identified.", self.styles['Normal']))
        
        elements.append(Spacer(1, 20))
        return elements
    
    def create_investment_timeline_section(self, assessment: RFPFeasibilityAssessment) -> List:
        """Create investment and timeline section"""
        elements = []
        
        elements.append(Paragraph("Investment & Timeline", self.heading_style))
        elements.append(Spacer(1, 12))
        
        inv_time_data = [
            ['Aspect', 'Details'],
            ['Investment Required', assessment.investment_required],
            ['Timeline Estimate', assessment.timeline_estimate],
            ['Resource Requirements', assessment.resource_requirements],
            ['ROI Expectation', 'Positive ROI within 6-12 months post-implementation'],
            ['Critical Path', 'Module development → Gap mitigation → Integration testing']
        ]
        
        inv_table = Table(inv_time_data, colWidths=[2*inch, 4*inch])
        inv_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTNAME', (0, 1), (0, -1), 'Helvetica-Bold'),
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f0f0f0')])
        ]))
        
        elements.append(inv_table)
        elements.append(Spacer(1, 20))
        return elements
    
    def create_win_loss_analysis_section(self, factors: List[WinLossFactor]) -> List:
        """Create win/loss analysis section"""
        elements = []
        
        elements.append(Paragraph("Historical Win/Loss Analysis", self.heading_style))
        elements.append(Spacer(1, 12))
        
        # Separate win and loss factors
        win_factors = [f for f in factors if f.factor_type == 'Win']
        loss_factors = [f for f in factors if f.factor_type == 'Loss']
        
        # Summary
        summary_text = f"Based on {len(factors)} historical factors: {len(win_factors)} win factors, {len(loss_factors)} loss factors"
        elements.append(Paragraph(summary_text, self.styles['Normal']))
        elements.append(Spacer(1, 12))
        
        # Win factors
        if win_factors:
            elements.append(Paragraph("Key Success Factors", self.subheading_style))
            elements.append(Spacer(1, 8))
            
            for i, factor in enumerate(win_factors[:5], 1):
                elements.append(Paragraph(f"{i}. [{factor.category}] {factor.description}", self.styles['Normal']))
                elements.append(Spacer(1, 4))
        
        elements.append(Spacer(1, 8))
        
        # Loss factors
        if loss_factors:
            elements.append(Paragraph("Common Loss Factors", self.subheading_style))
            elements.append(Spacer(1, 8))
            
            for i, factor in enumerate(loss_factors[:5], 1):
                elements.append(Paragraph(f"{i}. [{factor.category}] {factor.description}", self.styles['Normal']))
                elements.append(Spacer(1, 4))
        
        elements.append(Spacer(1, 20))
        return elements
    
    def create_requirements_coverage_section(self, requirements: List[Requirement]) -> List:
        """Create requirements coverage section"""
        elements = []
        
        elements.append(Paragraph("Requirements Coverage Analysis", self.heading_style))
        elements.append(Spacer(1, 12))
        
        if requirements:
            # Group by priority
            must_haves = [r for r in requirements if r.priority == 'Must Have']
            nice_to_haves = [r for r in requirements if r.priority == 'Nice to Have']
            optionals = [r for r in requirements if r.priority == 'Optional']
            
            # Summary
            summary_text = f"Total Requirements: {len(requirements)} | Must Have: {len(must_haves)} | Nice to Have: {len(nice_to_haves)} | Optional: {len(optionals)}"
            elements.append(Paragraph(summary_text, self.styles['Normal']))
            elements.append(Spacer(1, 12))
            
            # Requirements table
            req_data = [['ID', 'Type', 'Description', 'Priority', 'Our Capability', 'Gap']]
            
            # Sort by priority and capability
            sorted_reqs = sorted(requirements, key=lambda x: (
                0 if x.priority == 'Must Have' else 1 if x.priority == 'Nice to Have' else 2,
                0 if x.our_capability == 'None' else 1 if x.our_capability == 'Partial' else 2
            ))
            
            for req in sorted_reqs[:15]:  # Top 15 requirements
                capability_color = colors.green if req.our_capability == 'Full' else colors.orange if req.our_capability == 'Partial' else colors.red if req.our_capability == 'None' else colors.blue
                
                req_data.append([
                    req.requirement_type[:8],
                    Paragraph(req.description[:50] + '...' if len(req.description) > 50 else req.description, self.styles['Normal']),
                    req.priority,
                    Paragraph(req.our_capability, ParagraphStyle('Cap', parent=self.styles['Normal'], textColor=capability_color))
                ])
            
            req_table = Table(req_data, colWidths=[0.6*inch, 0.8*inch, 2.5*inch, 0.9*inch, 1*inch, 0.5*inch])
            req_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, -1), 8),
                ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f5f5f5')])
            ]))
            
            elements.append(req_table)
        else:
            elements.append(Paragraph("No requirements identified in the analysis.", self.styles['Normal']))
        
        elements.append(Spacer(1, 20))
        return elements
    
    def create_detailed_rationale_section(self, assessment: RFPFeasibilityAssessment) -> List:
        """Create detailed rationale section"""
        elements = []
        
        elements.append(Paragraph("Detailed Analysis Rationale", self.heading_style))
        elements.append(Spacer(1, 12))
        
        # Convert markdown-style formatting
        rationale_parts = assessment.detailed_rationale.split('\n')
        for part in rationale_parts:
            if part.startswith('##'):
                clean_text = part.replace('##', '').strip()
                elements.append(Paragraph(clean_text, self.subheading_style))
            elif part.startswith('###'):
                clean_text = part.replace('###', '').strip()
                elements.append(Paragraph(clean_text, ParagraphStyle(
                    'SubSubHeading',
                    parent=self.styles['Normal'],
                    fontName='Helvetica-Bold',
                    fontSize=10
                )))
            elif part.startswith('-'):
                clean_text = '• ' + part[1:].strip()
                elements.append(Paragraph(clean_text, self.styles['Normal']))
            elif part.strip():
                elements.append(Paragraph(part.strip(), self.styles['Normal']))
            
            if part.strip():
                elements.append(Spacer(1, 4))
        
        elements.append(Spacer(1, 20))
        return elements
    
    def create_appendix(self, analyzer: RFPFeasibilityAnalyzer) -> List:
        """Create appendix with additional details"""
        elements = []
        
        elements.append(PageBreak())
        elements.append(Paragraph("Appendix", self.heading_style))
        elements.append(Spacer(1, 12))
        
        # Document analysis summary
        elements.append(Paragraph("Document Analysis Summary", self.subheading_style))
        elements.append(Spacer(1, 8))
        
        doc_data = [['Document Type', 'Size (chars)', 'Sentences', 'Key Findings']]
        for doc_type, summary in analyzer.document_summaries.items():
            doc_data.append([
                doc_type.replace('_', ' ').title(),
                f"{summary['length']:,}",
                str(summary['sentences']),
                f"{len(analyzer.modules if doc_type == 'module_matching' else analyzer.gaps if doc_type == 'gap_analysis' else analyzer.requirements if doc_type == 'customer_needs' else analyzer.win_loss_factors)} items extracted"
            ])
        
        doc_table = Table(doc_data, colWidths=[1.5*inch, 1*inch, 1*inch, 2.5*inch])
        doc_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, -1), 9),
            ('VALIGN', (0, 0), (-1, -1), 'TOP')
        ]))
        
        elements.append(doc_table)
        elements.append(Spacer(1, 20))
        
        # Methodology note
        elements.append(Paragraph("Analysis Methodology", self.subheading_style))
        elements.append(Spacer(1, 8))
        methodology_text = """
        This feasibility assessment was generated using advanced text analysis and pattern recognition 
        techniques to extract structured information from unstructured documents. The analysis includes:
        
        • Module mapping and coverage assessment
        • Gap identification and severity classification
        • Requirements extraction and capability matching
        • Historical win/loss factor analysis
        • Risk assessment and mitigation planning
        • Investment and timeline estimation
        
        Confidence scores are calculated based on weighted factors including module coverage, 
        requirement fulfillment, gap severity, and historical performance metrics.
        """
        elements.append(Paragraph(methodology_text.strip(), self.styles['Normal']))
        
        return elements
    
    def create_ai_insights_section(self, assessment: RFPFeasibilityAssessment, analyzer: RFPFeasibilityAnalyzer) -> List:
        """Create AI insights section"""
        elements = []
        
        elements.append(Paragraph("AI-Generated Strategic Insights", self.heading_style))
        elements.append(Spacer(1, 12))
        
        # Document-level insights if available
        if any('llm_summary' in summary for summary in analyzer.document_summaries.values()):
            elements.append(Paragraph("Document Analysis Insights", self.subheading_style))
            elements.append(Spacer(1, 8))
            
            for doc_type, summary in analyzer.document_summaries.items():
                if 'llm_summary' in summary:
                    elements.append(Paragraph(f"<b>{doc_type.replace('_', ' ').title()}:</b>", self.styles['Normal']))
                    elements.append(Spacer(1, 4))
                    
                    # Format the LLM summary nicely
                    insights_text = summary['llm_summary']
                    if len(insights_text) > 500:
                        insights_text = insights_text[:497] + "..."
                    
                    elements.append(Paragraph(insights_text, ParagraphStyle(
                        'AIInsight',
                        parent=self.styles['Normal'],
                        fontSize=9,
                        leftIndent=20,
                        rightIndent=20,
                        textColor=colors.HexColor('#333333')
                    )))
                    elements.append(Spacer(1, 12))
        
        # Strategic recommendations from detailed rationale
        if "AI Module Analysis:" in assessment.detailed_rationale:
            elements.append(Paragraph("Strategic Analysis", self.subheading_style))
            elements.append(Spacer(1, 8))
            
            # Extract AI sections from detailed rationale
            rationale_parts = assessment.detailed_rationale.split('\n')
            capture_ai = False
            ai_content = []
            
            for part in rationale_parts:
                if "AI" in part and ("Analysis:" in part or "Plan:" in part):
                    capture_ai = True
                    ai_content.append(Paragraph(f"<b>{part.strip()}</b>", self.styles['Normal']))
                elif capture_ai and part.strip() and not part.startswith('#'):
                    ai_content.append(Paragraph(part.strip(), ParagraphStyle(
                        'AIContent',
                        parent=self.styles['Normal'],
                        fontSize=9,
                        leftIndent=10
                    )))
                elif part.startswith('#'):
                    capture_ai = False
            
            for element in ai_content[:10]:  # Limit to prevent too long
                elements.append(element)
                elements.append(Spacer(1, 4))
        
        elements.append(Spacer(1, 20))
        return elements
    
    def generate_report(self, analyzer: RFPFeasibilityAnalyzer, assessment: RFPFeasibilityAssessment, output_path: str):
        """Generate complete feasibility report"""
        doc = SimpleDocTemplate(
            output_path, 
            pagesize=A4,
            topMargin=0.75*inch,
            bottomMargin=0.75*inch,
            leftMargin=0.75*inch,
            rightMargin=0.75*inch
        )
        elements = []
        
        # Title page
        elements.append(Spacer(1, 1*inch))
        elements.append(Paragraph("RFP Response Feasibility Analysis", self.title_style))
        elements.append(Spacer(1, 20))
        elements.append(Paragraph(f"Generated on {datetime.now().strftime('%B %d, %Y at %I:%M %p')}", 
                                  ParagraphStyle('DateStyle', parent=self.styles['Normal'], alignment=TA_CENTER)))
        elements.append(Spacer(1, 40))
        
        # Add document count summary
        doc_count = len(analyzer.document_summaries)
        total_chars = sum(d['length'] for d in analyzer.document_summaries.values())
        elements.append(Paragraph(f"Analysis based on {doc_count} documents containing {total_chars:,} characters", 
                                  ParagraphStyle('Summary', parent=self.styles['Normal'], alignment=TA_CENTER, fontSize=10)))
        elements.append(PageBreak())
        
        # Table of Contents
        elements.append(Paragraph("Table of Contents", self.heading_style))
        elements.append(Spacer(1, 12))
        toc_items = [
            "1. GO/NO-GO Decision",
            "2. Executive Summary",
            "3. AI-Generated Strategic Insights",
            "4. Module Analysis",
            "5. Gap Analysis",
            "6. Requirements Coverage",
            "7. Historical Win/Loss Analysis",
            "8. SWOT Analysis",
            "9. Risk Assessment",
            "10. Action Plan",
            "11. Investment & Timeline",
            "12. Detailed Rationale",
            "13. Competitive Positioning",
            "14. Appendix"
        ]
        for item in toc_items:
            elements.append(Paragraph(item, self.styles['Normal']))
            elements.append(Spacer(1, 6))
        elements.append(PageBreak())
        
        # Main sections
        elements.extend(self.create_go_no_go_decision(assessment))
        elements.append(PageBreak())
        
        elements.extend(self.create_executive_summary(assessment))
        
        # Add AI insights section
        elements.extend(self.create_ai_insights_section(assessment, analyzer))
        
        elements.extend(self.create_module_analysis_section(analyzer.modules, assessment))
        
        elements.extend(self.create_gap_analysis_section(assessment))
        
        elements.extend(self.create_requirements_coverage_section(analyzer.requirements))
        
        elements.extend(self.create_win_loss_analysis_section(analyzer.win_loss_factors))
        
        elements.extend(self.create_swot_analysis(assessment))
        
        elements.extend(self.create_risk_assessment_section(assessment))
        
        elements.extend(self.create_action_plan(assessment))
        
        elements.extend(self.create_investment_timeline_section(assessment))
        
        elements.extend(self.create_detailed_rationale_section(assessment))
        
        # Competitive positioning
        elements.append(Paragraph("Competitive Positioning", self.heading_style))
        elements.append(Spacer(1, 12))
        elements.append(Paragraph(assessment.competitive_positioning, self.styles['Normal']))
        elements.append(Spacer(1, 20))
        
        # Appendix
        elements.extend(self.create_appendix(analyzer))
        
        # Build PDF
        try:
            doc.build(elements)
            print(f"✅ RFP Feasibility Report generated successfully: {output_path}")
        except Exception as e:
            print(f"❌ Error generating PDF: {e}")


In [178]:
class RFPResponseGenerator:
    """Generates comprehensive RFP responses based on feasibility analysis"""
    
    def __init__(self, llm_processor: LLMProcessor):
        self.llm_processor = llm_processor
        
    def generate_technical_proposal(self, modules: List[ModuleMapping], gaps: List[GapItem], 
                                  requirements: List[Requirement]) -> TechnicalProposal:
        """Generate detailed technical proposal with improved structure"""
        
        # Categorize modules for better presentation
        available_modules = [m for m in modules if m.status == "Available"]
        partial_modules = [m for m in modules if m.status == "Partial"]
        missing_modules = [m for m in modules if m.status == "Missing"]
        
        # Enhanced Solution Overview
        solution_prompt = f"""
        Create a professional solution overview for an RFP response. Our capabilities include:
        
        AVAILABLE MODULES ({len(available_modules)}):
        {chr(10).join([f'- {m.module_name}: {m.notes.split("|")[0] if "|" in m.notes else m.notes[:60]}' for m in available_modules])}
        
        MODULES REQUIRING DEVELOPMENT ({len(missing_modules)}):
        {chr(10).join([f'- {m.module_name}' for m in missing_modules])}
        
        CLIENT REQUIREMENTS:
        {chr(10).join([f'- {r.description[:80]}' for r in requirements[:3]])}
        
        Write a compelling solution overview (200-250 words) that:
        1. Positions our existing capabilities as competitive advantages
        2. Demonstrates understanding of client needs
        3. Shows how we'll address any gaps
        4. Emphasizes our technical expertise and proven approach
        """
        solution_overview = self.llm_processor._generate_text(solution_prompt, 500)
        
        # Enhanced Technical Architecture
        arch_prompt = f"""
        Design a comprehensive technical architecture description for our solution:
        
        CORE SYSTEM COMPONENTS:
        - {len(available_modules)} proven modules ready for deployment
        - {len(partial_modules)} modules requiring customization
        - {len(missing_modules)} new modules for development
        
        INTEGRATION REQUIREMENTS:
        - Seamless data flow between all modules
        - API-first architecture for extensibility
        - Cloud-native deployment with on-premise options
        
        Write a technical architecture section (180-220 words) covering:
        1. High-level system design and component interaction
        2. Data management and storage strategy
        3. Security and performance considerations
        4. Technology stack and frameworks
        """
        technical_architecture = self.llm_processor._generate_text(arch_prompt, 450)
        
        # Enhanced Implementation Approach
        total_complexity_score = len(missing_modules) * 3 + len(partial_modules) * 1.5 + len([g for g in gaps if g.severity == 'High']) * 2
        
        impl_prompt = f"""
        Create a structured implementation approach for a project with complexity score {total_complexity_score:.1f}:
        
        PROJECT SCOPE:
        - Configure {len(available_modules)} existing modules
        - Customize {len(partial_modules)} partial modules  
        - Develop {len(missing_modules)} new modules from scratch
        - Address {len([g for g in gaps if g.severity in ['High', 'Critical']])} critical gaps
        
        Write an implementation approach (160-200 words) with:
        1. Phased delivery strategy (3-4 phases)
        2. Risk mitigation through proven methodologies
        3. Quality assurance at each milestone
        4. Client collaboration and feedback loops
        """
        implementation_approach = self.llm_processor._generate_text(impl_prompt, 400)
        
        # Enhanced Integration Strategy
        integration_prompt = f"""
        Define integration strategy for a solution connecting {len(modules)} modules:
        
        INTEGRATION CHALLENGES:
        - Data format standardization across modules
        - Real-time synchronization requirements
        - Legacy system compatibility
        - Third-party API integration
        
        Write integration strategy (120-150 words) addressing:
        1. API design principles and standards
        2. Data transformation and validation
        3. Error handling and monitoring
        4. Performance optimization techniques
        """
        integration_strategy = self.llm_processor._generate_text(integration_prompt, 300)
        
        # Enhanced Security Framework
        security_prompt = f"""
        Design enterprise security framework for a solution handling sensitive data:
        
        SECURITY REQUIREMENTS:
        - Multi-layer security architecture
        - Compliance with industry standards (ISO 27001, SOC 2)
        - Data encryption at rest and in transit
        - Role-based access control
        
        Write security framework description (140-170 words) covering:
        1. Authentication and authorization mechanisms
        2. Data protection and privacy controls
        3. Security monitoring and incident response
        4. Compliance and audit capabilities
        """
        security_framework = self.llm_processor._generate_text(security_prompt, 350)
        
        # Enhanced Performance Specifications
        perf_specs = f"""
        **Performance Specifications & Commitments:**
        
        **Response Time Performance:**
        - Standard Operations: < 1.5 seconds (target: 1.2 seconds)
        - Complex Queries: < 3.0 seconds (target: 2.5 seconds)
        - Report Generation: < 5.0 seconds for standard reports
        
        **Scalability & Capacity:**
        - Concurrent Users: 2,000+ simultaneous users (scalable to 5,000+)
        - Data Processing: 15GB/hour sustained (burst to 25GB/hour)
        - Storage Capacity: Petabyte-scale with auto-scaling
        
        **Availability & Reliability:**
        - System Uptime: 99.95% with SLA guarantees
        - Disaster Recovery: < 4 hours RTO, < 1 hour RPO
        - Load Balancing: Auto-scaling based on demand
        
        **Quality Metrics:**
        - Data Accuracy: 99.9% with validation checks
        - Security Response: < 15 minutes for critical incidents
        - Backup Frequency: Continuous with point-in-time recovery
        """
        
        # Enhanced Testing Methodology
        testing_methodology = f"""
        **Comprehensive Testing Framework:**
        
        **Phase 1: Foundation Testing**
        - Unit Testing: 90%+ code coverage for all modules
        - Component Testing: Individual module validation
        - Integration Testing: API and data flow verification
        
        **Phase 2: System Testing**
        - Performance Testing: Load testing with 150% expected capacity
        - Security Testing: Penetration testing and vulnerability assessment
        - Compatibility Testing: Cross-platform and browser validation
        
        **Phase 3: Acceptance Testing**
        - User Acceptance Testing: Client-driven scenario validation
        - Regression Testing: Automated test suite execution
        - Production Readiness: Go-live checklist verification
        
        **Continuous Quality Assurance:**
        - Automated Testing: CI/CD pipeline integration
        - Code Review: Peer review for all changes
        - Quality Gates: Mandatory checkpoints at each phase
        """
        
        # Enhanced Deployment Strategy
        deployment_prompt = f"""
        Create deployment strategy for enterprise solution with {len(modules)} modules:
        
        DEPLOYMENT CONSIDERATIONS:
        - Minimal business disruption during rollout
        - Rollback capabilities for risk mitigation
        - Production monitoring and alerting
        - User training and change management
        
        Write deployment strategy (130-160 words) including:
        1. Blue-green deployment approach
        2. Phased rollout with pilot groups
        3. Monitoring and health checks
        4. Post-deployment support procedures
        """
        deployment_strategy = self.llm_processor._generate_text(deployment_prompt, 320)
        
        return TechnicalProposal(
            solution_overview=solution_overview or "Our comprehensive solution leverages proven modules and industry expertise to deliver exceptional value and performance.",
            technical_architecture=technical_architecture or "Scalable, cloud-native architecture designed for enterprise-grade performance, security, and reliability.",
            implementation_approach=implementation_approach or "Agile, phased implementation approach ensuring minimal risk and maximum value delivery at each milestone.",
            integration_strategy=integration_strategy or "API-first integration strategy ensuring seamless data flow and system interoperability.",
            security_framework=security_framework or "Multi-layered security framework with enterprise-grade encryption, access controls, and compliance capabilities.",
            performance_specifications=perf_specs,
            testing_methodology=testing_methodology,
            deployment_strategy=deployment_strategy or "Blue-green deployment strategy with comprehensive monitoring and rollback capabilities."
        )
    
    def generate_financial_proposal(self, modules: List[ModuleMapping], gaps: List[GapItem], 
                                  assessment: RFPFeasibilityAssessment) -> FinancialProposal:
        """Generate detailed financial proposal"""
        
        # Calculate costs based on modules and gaps
        available_modules = [m for m in modules if m.status == "Available"]
        partial_modules = [m for m in modules if m.status == "Partial"]
        missing_modules = [m for m in modules if m.status == "Missing"]
        critical_gaps = [g for g in gaps if g.severity in ['High', 'Critical']]
        
        # Base cost calculation
        base_cost = len(available_modules) * 10000  # $10K per existing module
        development_cost = len(missing_modules) * 75000  # $75K per new module
        customization_cost = len(partial_modules) * 25000  # $25K per partial module
        gap_resolution_cost = len(critical_gaps) * 15000  # $15K per critical gap
        
        total_project_cost = base_cost + development_cost + customization_cost + gap_resolution_cost
        
        # Cost breakdown
        cost_breakdown = {
            "Existing Module Configuration": f"${base_cost:,}",
            "New Module Development": f"${development_cost:,}",
            "Module Customization": f"${customization_cost:,}",
            "Gap Resolution": f"${gap_resolution_cost:,}",
            "Project Management (15%)": f"${int(total_project_cost * 0.15):,}",
            "Testing & QA (10%)": f"${int(total_project_cost * 0.10):,}",
            "Training & Documentation (8%)": f"${int(total_project_cost * 0.08):,}",
            "Contingency (5%)": f"${int(total_project_cost * 0.05):,}"
        }
        
        # Total with overhead
        total_with_overhead = total_project_cost * 1.38  # 38% total overhead
        
        # Payment schedule
        payment_schedule = f"""
        Payment Schedule:
        - Contract Signing: 20% (${int(total_with_overhead * 0.20):,})
        - Design Approval: 25% (${int(total_with_overhead * 0.25):,})
        - Development Milestone 1: 20% (${int(total_with_overhead * 0.20):,})
        - Development Milestone 2: 20% (${int(total_with_overhead * 0.20):,})
        - UAT Completion: 10% (${int(total_with_overhead * 0.10):,})
        - Go-Live: 5% (${int(total_with_overhead * 0.05):,})
        """
        
        # ROI Analysis
        roi_prompt = f"""
        Calculate ROI for a ${int(total_with_overhead):,} investment that will:
        - Improve efficiency by 30%
        - Reduce operational costs by 20%
        - Enable new revenue streams
        
        Provide ROI analysis (150 words) with specific metrics and timeframes.
        """
        roi_analysis = self.llm_processor._generate_text(roi_prompt, 300)
        
        # Cost justification
        cost_justification = f"""
        Investment Justification:
        This ${int(total_with_overhead):,} investment delivers:
        - {len(available_modules)} proven, enterprise-ready modules
        - {len(missing_modules)} custom-developed solutions
        - Comprehensive testing and quality assurance
        - Full documentation and training
        - 12 months warranty and support
        
        Cost per module: ${int(total_with_overhead / max(len(modules), 1)):,}
        Industry benchmark: 15-20% below market rates
        """
        
        # Pricing model
        pricing_model = """
        Flexible Pricing Options:
        1. Fixed Price: Total project cost with defined scope
        2. Time & Materials: $150-200/hour for additional work
        3. Milestone-based: Payments tied to deliverable completion
        4. Hybrid: Fixed core + T&M for enhancements
        """
        
        # Maintenance costs
        annual_maintenance = int(total_with_overhead * 0.18)  # 18% annually
        maintenance_costs = f"""
        Annual Maintenance & Support: ${annual_maintenance:,}
        Includes:
        - 24/7 technical support
        - Software updates and patches
        - Performance monitoring
        - Security updates
        - Documentation updates
        """
        
        # Optional services
        optional_services = {
            "Advanced Analytics Module": "$45,000",
            "Mobile Application": "$35,000",
            "API Gateway Enhancement": "$25,000",
            "Additional Training (per day)": "$2,500",
            "Custom Reporting Engine": "$30,000"
        }
        
        return FinancialProposal(
            total_cost=f"${int(total_with_overhead):,}",
            cost_breakdown=cost_breakdown,
            payment_schedule=payment_schedule,
            roi_analysis=roi_analysis or "Expected ROI of 250% within 24 months through efficiency gains.",
            cost_justification=cost_justification,
            pricing_model=pricing_model,
            maintenance_costs=maintenance_costs,
            optional_services=optional_services
        )
    
    def generate_project_timeline(self, modules: List[ModuleMapping], gaps: List[GapItem]) -> str:
        """Generate detailed project timeline"""
        
        missing_modules = [m for m in modules if m.status == "Missing"]
        critical_gaps = [g for g in gaps if g.severity in ['High', 'Critical']]
        
        # Calculate timeline based on complexity
        base_weeks = 12  # Base project duration
        development_weeks = len(missing_modules) * 3  # 3 weeks per new module
        gap_weeks = len(critical_gaps) * 1  # 1 week per critical gap
        
        total_weeks = base_weeks + development_weeks + gap_weeks
        
        timeline = f"""
        Project Timeline: {total_weeks} weeks total
        
        Phase 1: Project Initiation (Weeks 1-2)
        - Project kickoff and team mobilization
        - Requirements validation and finalization
        - Technical architecture review
        
        Phase 2: Design & Planning (Weeks 3-4)
        - Detailed system design
        - Database schema design
        - Integration architecture
        - Test plan development
        
        Phase 3: Development (Weeks 5-{5 + development_weeks})
        - Core module development/configuration
        - Custom module development ({len(missing_modules)} modules)
        - Gap resolution implementation
        - Continuous integration setup
        
        Phase 4: Integration & Testing (Weeks {6 + development_weeks}-{8 + development_weeks})
        - System integration
        - Performance testing
        - Security testing
        - User acceptance testing
        
        Phase 5: Deployment & Go-Live (Weeks {9 + development_weeks}-{total_weeks})
        - Production deployment
        - Data migration
        - User training
        - Go-live support
        
        Parallel Activities:
        - Documentation (ongoing)
        - Quality assurance (ongoing)
        - Client communication (weekly)
        """
        
        return timeline
    
    def generate_comprehensive_response(self, analyzer: RFPFeasibilityAnalyzer, 
                                      assessment: RFPFeasibilityAssessment) -> RFPResponse:
        """Generate complete RFP response"""
        
        # Executive Summary
        exec_summary_prompt = f"""
        Write an executive summary for an RFP response with:
        - Confidence score: {assessment.confidence_score:.1f}%
        - {assessment.available_modules}/{assessment.total_modules} modules available
        - {len(assessment.critical_gaps)} critical gaps
        - Investment: {assessment.investment_required}
        
        Create compelling executive summary (200 words) highlighting our strengths and approach.
        """
        executive_summary = self.llm_processor._generate_text(exec_summary_prompt, 400)
        
        # Generate technical and financial proposals
        technical_proposal = self.generate_technical_proposal(
            analyzer.modules, analyzer.gaps, analyzer.requirements
        )
        
        financial_proposal = self.generate_financial_proposal(
            analyzer.modules, analyzer.gaps, assessment
        )
        
        # Project timeline
        project_timeline = self.generate_project_timeline(analyzer.modules, analyzer.gaps)
        
        # Risk mitigation
        risk_mitigation = f"""
        Risk Mitigation Strategy:
        
        Identified Risks: {len(assessment.risks)}
        
        Mitigation Measures:
        1. Technical Risk: Proven module architecture reduces implementation risk
        2. Timeline Risk: Agile methodology with regular checkpoints
        3. Budget Risk: Fixed-price model with defined scope
        4. Quality Risk: Comprehensive testing at each phase
        5. Resource Risk: Dedicated team with backup resources
        
        Contingency Plans:
        - 5% budget contingency for scope changes
        - Alternative implementation paths for critical modules
        - Escalation procedures for issue resolution
        """
        
        # Team composition
        team_composition = f"""
        Dedicated Project Team:
        
        Core Team ({4 + len([m for m in analyzer.modules if m.status == "Missing"])//2} members):
        - Project Manager (PMP certified)
        - Solution Architect (10+ years experience)
        - Lead Developer (Full-stack expertise)
        - QA Manager (Testing specialist)
        {"- Additional Developers (2-3 for module development)" if len([m for m in analyzer.modules if m.status == "Missing"]) > 2 else ""}
        
        Extended Team:
        - Business Analyst
        - UI/UX Designer
        - DevOps Engineer
        - Security Specialist
        
        Team Qualifications:
        - Average 8+ years industry experience
        - Relevant technology certifications
        - Previous similar project success
        """
        
        # Compliance statement
        compliance_statement = """
        Compliance & Standards:
        
        Our solution ensures compliance with:
        - Industry security standards (ISO 27001, SOC 2)
        - Data protection regulations (GDPR, CCPA)
        - Accessibility standards (WCAG 2.1)
        - Quality standards (ISO 9001)
        
        Certifications:
        - Company ISO certifications
        - Team member professional certifications
        - Technology partner certifications
        """
        
        # References and case studies
        references_prompt = f"""
        Create references section for RFP response highlighting:
        - Similar projects completed successfully
        - Client testimonials
        - Relevant case studies
        
        Write references section (150 words) with specific examples.
        """
        references_case_studies = self.llm_processor._generate_text(references_prompt, 300)
        
        # Assumptions and dependencies
        assumptions_dependencies = f"""
        Project Assumptions:
        - Client provides timely access to existing systems
        - Stakeholders available for requirements validation
        - Test environment provided within 2 weeks
        - Decision-making authority clearly defined
        
        Dependencies:
        - Third-party system APIs available as documented
        - Network infrastructure meets minimum requirements
        - Security approvals obtained within agreed timeframes
        - User training schedule coordinated with deployment
        
        Success Criteria:
        - All {len(analyzer.requirements)} requirements fully met
        - Performance benchmarks achieved
        - User acceptance criteria satisfied
        - Go-live within agreed timeline
        """
        
        return RFPResponse(
            executive_summary=executive_summary or "We are uniquely positioned to deliver this solution with our proven expertise and comprehensive approach.",
            technical_proposal=technical_proposal,
            financial_proposal=financial_proposal,
            project_timeline=project_timeline,
            risk_mitigation=risk_mitigation,
            team_composition=team_composition,
            compliance_statement=compliance_statement,
            references_case_studies=references_case_studies or "Our track record includes successful implementations for Fortune 500 companies with similar requirements.",
            assumptions_dependencies=assumptions_dependencies
        )


In [179]:
class RFPResponseReportGenerator:
    """Generates professional RFP response document"""
    
    def __init__(self):
        self.styles = getSampleStyleSheet()
        self.title_style = ParagraphStyle(
            'RFPTitle',
            parent=self.styles['Heading1'],
            fontSize=28,
            spaceAfter=30,
            alignment=TA_CENTER,
            textColor=colors.HexColor('#1e3a8a')
        )
        self.section_style = ParagraphStyle(
            'RFPSection',
            parent=self.styles['Heading2'],
            fontSize=18,
            spaceAfter=15,
            textColor=colors.HexColor('#1e3a8a'),
            borderWidth=1,
            borderColor=colors.HexColor('#1e3a8a'),
            borderPadding=8,
            backColor=colors.HexColor('#f8fafc')
        )
        
    def create_cover_page(self, rfp_response) -> List:
        """Create professional cover page"""
        elements = []
        
        elements.append(Spacer(1, 2*inch))
        elements.append(Paragraph("PROPOSAL RESPONSE", self.title_style))
        elements.append(Spacer(1, 0.5*inch))
        
        # Company info box
        company_info = """
        <b>Submitted by:</b> Abougia Technologies<br/>
        <b>Date:</b> {}<br/>
        <b>Proposal ID:</b> RFP-2025-001<br/>
        <b>Validity:</b> 90 days from submission<br/>
        <b>Contact:</b> proposals@abougia.com
        """.format(datetime.now().strftime('%B %d, %Y'))
        
        elements.append(Paragraph(company_info, ParagraphStyle(
            'CompanyInfo',
            parent=self.styles['Normal'],
            fontSize=12,
            alignment=TA_CENTER,
            borderWidth=2,
            borderColor=colors.HexColor('#1e3a8a'),
            borderPadding=20,
            backColor=colors.HexColor('#f0f9ff')
        )))
        
        elements.append(Spacer(1, 1*inch))
        elements.append(Paragraph("Confidential & Proprietary", ParagraphStyle(
            'Confidential',
            parent=self.styles['Normal'],
            fontSize=10,
            alignment=TA_CENTER,
            textColor=colors.red
        )))
        
        elements.append(PageBreak())
        return elements
    
    def create_executive_summary_section(self, rfp_response) -> List:
        """Create enhanced executive summary section"""
        elements = []
        
        elements.append(Paragraph("EXECUTIVE SUMMARY", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Enhanced executive summary with structured approach
        summary_intro = """
        <b>Abougia Technologies</b> is pleased to submit this comprehensive proposal in response to your RFP. 
        Our solution leverages proven technology modules, industry expertise, and innovative approaches to 
        deliver exceptional value that exceeds your requirements.
        """
        
        elements.append(Paragraph(summary_intro, ParagraphStyle(
            'SummaryIntro',
            parent=self.styles['Normal'],
            fontSize=11,
            spaceAfter=12,
            backColor=colors.HexColor('#f8fafc'),
            borderPadding=10,
            borderWidth=1,
            borderColor=colors.HexColor('#e2e8f0')
        )))
        
        elements.append(Spacer(1, 12))
        
        # Process the AI-generated executive summary with better formatting
        ai_summary = rfp_response.executive_summary
        if ai_summary and len(ai_summary) > 100:
            # Split into logical paragraphs
            summary_sentences = ai_summary.split('. ')
            current_paragraph = ""
            
            for sentence in summary_sentences:
                if len(current_paragraph + sentence) > 180 and current_paragraph:
                    elements.append(Paragraph(current_paragraph.strip() + '.', self.styles['Normal']))
                    elements.append(Spacer(1, 8))
                    current_paragraph = sentence
                else:
                    current_paragraph += sentence + '. ' if not sentence.endswith('.') else sentence + ' '
            
            if current_paragraph.strip():
                elements.append(Paragraph(current_paragraph.strip(), self.styles['Normal']))
        else:
            # Fallback professional summary
            fallback_summary = """
            Our proposed solution addresses your core requirements through a combination of proven modules 
            and custom development. We bring extensive experience in similar implementations, a dedicated 
            project team, and a commitment to delivering on time and within budget.
            
            Key differentiators include our modular architecture approach, comprehensive testing methodology, 
            and ongoing support framework that ensures long-term success of your investment.
            """
            elements.append(Paragraph(fallback_summary, self.styles['Normal']))
        
        elements.append(Spacer(1, 15))
        
        # Enhanced Key highlights box with better structure
        highlights_header = Paragraph("<b>Key Proposal Highlights:</b>", ParagraphStyle(
            'HighlightHeader',
            parent=self.styles['Normal'],
            fontName='Helvetica-Bold',
            fontSize=12,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        ))
        
        highlights_content = [
            "✓ <b>Proven Solution:</b> Leveraging existing modules with demonstrated success",
            "✓ <b>Technical Excellence:</b> Comprehensive architecture with scalability built-in", 
            "✓ <b>Competitive Investment:</b> Fixed-price model with transparent cost breakdown",
            "✓ <b>Expert Team:</b> Dedicated professionals with relevant domain expertise",
            "✓ <b>Risk Mitigation:</b> Phased delivery approach with defined milestones",
            "✓ <b>Long-term Partnership:</b> Comprehensive warranty and ongoing support"
        ]
        
        elements.append(highlights_header)
        
        # Add summary box around highlights
        highlight_table_data = []
        for highlight in highlights_content:
            highlight_table_data.append([Paragraph(highlight, ParagraphStyle(
                'HighlightTable',
                parent=self.styles['Normal'],
                fontSize=10
            ))])
        
        highlight_table = Table(highlight_table_data, colWidths=[5.5*inch])
        highlight_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, -1), colors.HexColor('#f0f9ff')),
            ('BORDER', (0, 0), (-1, -1), 1, colors.HexColor('#1e3a8a')),
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('TOPPADDING', (0, 0), (-1, -1), 8),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
            ('LEFTPADDING', (0, 0), (-1, -1), 15),
            ('RIGHTPADDING', (0, 0), (-1, -1), 15),
        ]))
        
        elements.append(highlight_table)
        elements.append(PageBreak())
        return elements

    def create_technical_section(self, technical_proposal) -> List:
        """Create enhanced technical proposal section with better formatting"""
        elements = []
        
        elements.append(Paragraph("TECHNICAL PROPOSAL", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Solution Overview with better formatting
        elements.append(Paragraph("1. Solution Overview", ParagraphStyle(
            'TechSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        )))
        
        # Split the solution overview into readable paragraphs
        solution_text = technical_proposal.solution_overview
        if "\n\n" in solution_text:
            # Already has paragraph breaks
            paragraphs = solution_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    elements.append(Paragraph(para.strip(), self.styles['Normal']))
                    elements.append(Spacer(1, 8))
        else:
            # Split into smaller paragraphs for better readability
            sentences = solution_text.split('. ')
            current_para = ""
            for sentence in sentences:
                if len(current_para + sentence) > 200 and current_para:
                    elements.append(Paragraph(current_para.strip() + '.', self.styles['Normal']))
                    elements.append(Spacer(1, 8))
                    current_para = sentence
                else:
                    current_para += sentence + '. ' if not sentence.endswith('.') else sentence + ' '
            if current_para.strip():
                elements.append(Paragraph(current_para.strip(), self.styles['Normal']))
        
        elements.append(Spacer(1, 15))
        
        # Technical Architecture
        elements.append(Paragraph("2. Technical Architecture", ParagraphStyle(
            'TechSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        )))
        
        # Format architecture with paragraph breaks
        architecture_text = technical_proposal.technical_architecture
        if "\n\n" in architecture_text:
            paragraphs = architecture_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    elements.append(Paragraph(para.strip(), self.styles['Normal']))
                    elements.append(Spacer(1, 8))
        else:
            elements.append(Paragraph(architecture_text, self.styles['Normal']))
        
        elements.append(Spacer(1, 15))
        
        # Implementation Approach
        elements.append(Paragraph("3. Implementation Methodology", ParagraphStyle(
            'TechSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        )))
        
        # Format implementation with paragraph breaks
        implementation_text = technical_proposal.implementation_approach
        if "\n\n" in implementation_text:
            paragraphs = implementation_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    elements.append(Paragraph(para.strip(), self.styles['Normal']))
                    elements.append(Spacer(1, 8))
        else:
            elements.append(Paragraph(implementation_text, self.styles['Normal']))
        
        elements.append(Spacer(1, 15))
        
        # Integration Strategy
        elements.append(Paragraph("4. Integration Strategy", ParagraphStyle(
            'TechSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        )))
        
        # Format integration with paragraph breaks
        integration_text = technical_proposal.integration_strategy
        if "\n\n" in integration_text:
            paragraphs = integration_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    elements.append(Paragraph(para.strip(), self.styles['Normal']))
                    elements.append(Spacer(1, 8))
        else:
            elements.append(Paragraph(integration_text, self.styles['Normal']))
        
        elements.append(Spacer(1, 15))
        
        # Security Framework
        elements.append(Paragraph("5. Security Framework", ParagraphStyle(
            'TechSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        )))
        
        # Format security with paragraph breaks
        security_text = technical_proposal.security_framework
        if "\n\n" in security_text:
            paragraphs = security_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    elements.append(Paragraph(para.strip(), self.styles['Normal']))
                    elements.append(Spacer(1, 8))
        else:
            elements.append(Paragraph(security_text, self.styles['Normal']))
        
        elements.append(Spacer(1, 15))
        
        # Performance Specifications with enhanced table
        elements.append(Paragraph("6. Performance Specifications", ParagraphStyle(
            'TechSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        )))
        
        # Parse performance specs if they contain structured data
        if "**" in technical_proposal.performance_specifications:
            # Handle enhanced format
            perf_lines = technical_proposal.performance_specifications.split('\n')
            for line in perf_lines:
                if line.strip():
                    if line.startswith('**') and line.endswith('**'):
                        # Bold header
                        clean_line = line.replace('**', '')
                        elements.append(Paragraph(clean_line, ParagraphStyle(
                            'PerfHeader',
                            parent=self.styles['Normal'],
                            fontName='Helvetica-Bold',
                            fontSize=10,
                            spaceAfter=4
                        )))
                    elif line.startswith('- '):
                        # Bullet point
                        elements.append(Paragraph(line, ParagraphStyle(
                            'PerfBullet',
                            parent=self.styles['Normal'],
                            fontSize=9,
                            leftIndent=20,
                            spaceAfter=2
                        )))
                    else:
                        elements.append(Paragraph(line, self.styles['Normal']))
        else:
            # Fallback to original table format
            perf_data = [
                ['Metric', 'Specification', 'Our Commitment'],
                ['Response Time', '< 2 seconds', '< 1.5 seconds average'],
                ['Concurrent Users', '1000+ users', '2000+ users supported'],
                ['Uptime', '99.9%', '99.95% with SLA'],
                ['Data Processing', '10GB/hour', '15GB/hour capacity'],
                ['Scalability', '5x load increase', '10x scaling capability']
            ]
            
            perf_table = Table(perf_data, colWidths=[2*inch, 2*inch, 2*inch])
            perf_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, -1), 9),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f5f5f5')])
            ]))
            
            elements.append(perf_table)
        
        elements.append(Spacer(1, 15))
        
        # Testing Methodology
        elements.append(Paragraph("7. Testing Methodology", ParagraphStyle(
            'TechSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        )))
        
        # Format testing methodology for better readability
        testing_lines = technical_proposal.testing_methodology.split('\n')
        for line in testing_lines:
            if line.strip():
                if line.startswith('**') and line.endswith('**'):
                    # Bold header
                    clean_line = line.replace('**', '')
                    elements.append(Paragraph(clean_line, ParagraphStyle(
                        'TestHeader',
                        parent=self.styles['Normal'],
                        fontName='Helvetica-Bold',
                        fontSize=11,
                        spaceAfter=6
                    )))
                elif line.startswith('- '):
                    # Bullet point
                    elements.append(Paragraph(line, ParagraphStyle(
                        'TestBullet',
                        parent=self.styles['Normal'],
                        fontSize=9,
                        leftIndent=20,
                        spaceAfter=3
                    )))
                elif line.strip().startswith(('1.', '2.', '3.', '4.', '5.', '6.')):
                    # Numbered list
                    elements.append(Paragraph(line, ParagraphStyle(
                        'TestNumber',
                        parent=self.styles['Normal'],
                        fontSize=9,
                        leftIndent=15,
                        spaceAfter=3
                    )))
                else:
                    elements.append(Paragraph(line, self.styles['Normal']))
        
        elements.append(Spacer(1, 15))
        
        # Deployment Strategy
        elements.append(Paragraph("8. Deployment Strategy", ParagraphStyle(
            'TechSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=8
        )))
        
        # Format deployment with paragraph breaks
        deployment_text = technical_proposal.deployment_strategy
        if "\n\n" in deployment_text:
            paragraphs = deployment_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    elements.append(Paragraph(para.strip(), self.styles['Normal']))
                    elements.append(Spacer(1, 8))
        else:
            elements.append(Paragraph(deployment_text, self.styles['Normal']))
        
        elements.append(PageBreak())
        return elements

    def create_financial_section(self, financial_proposal) -> List:
        """Create enhanced financial proposal section"""
        elements = []
        
        elements.append(Paragraph("FINANCIAL PROPOSAL", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Enhanced Total Investment presentation
        total_cost_clean = financial_proposal.total_cost.replace('$', '').replace(',', '')
        try:
            cost_value = int(total_cost_clean)
            formatted_cost = f"${cost_value:,}"
        except:
            formatted_cost = financial_proposal.total_cost
        
        elements.append(Paragraph(f"Total Project Investment: {formatted_cost}", ParagraphStyle(
            'TotalCost',
            parent=self.styles['Heading2'],
            fontSize=22,
            textColor=colors.HexColor('#059669'),
            alignment=TA_CENTER,
            borderWidth=3,
            borderColor=colors.HexColor('#059669'),
            borderPadding=15,
            backColor=colors.HexColor('#f0fdf4'),
            spaceAfter=20
        )))
        
        elements.append(Spacer(1, 15))
        
        # Investment Summary Box
        investment_summary = """
        <b>Investment Overview:</b><br/>
        • Comprehensive solution with proven ROI<br/>
        • Fixed-price model eliminates budget uncertainty<br/>
        • Flexible payment schedule aligned with deliverables<br/>
        • Competitive pricing with exceptional value proposition
        """
        
        elements.append(Paragraph(investment_summary, ParagraphStyle(
            'InvestmentSummary',
            parent=self.styles['Normal'],
            fontSize=10,
            borderWidth=1,
            borderColor=colors.HexColor('#059669'),
            borderPadding=12,
            backColor=colors.HexColor('#f0fdf4'),
            spaceAfter=15
        )))
        
        # Enhanced Cost Breakdown Table
        elements.append(Paragraph("Detailed Cost Breakdown", ParagraphStyle(
            'FinSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=10
        )))
        
        cost_data = [['Cost Component', 'Amount', 'Percentage']]
        
        # Calculate total for percentage calculation
        total_amount = 0
        cost_values = {}
        for component, cost in financial_proposal.cost_breakdown.items():
            clean_cost = cost.replace('$', '').replace(',', '')
            try:
                cost_values[component] = int(clean_cost)
                total_amount += cost_values[component]
            except:
                cost_values[component] = 0
        
        for component, cost in financial_proposal.cost_breakdown.items():
            percentage = f"{(cost_values[component] / total_amount * 100):.1f}%" if total_amount > 0 else "0%"
            cost_data.append([
                Paragraph(component, self.styles['Normal']),
                Paragraph(cost, ParagraphStyle('CostAmount', parent=self.styles['Normal'], alignment=TA_RIGHT)),
                Paragraph(percentage, ParagraphStyle('CostPercent', parent=self.styles['Normal'], alignment=TA_CENTER))
            ])
        
        # Add total row
        cost_data.append([
            Paragraph("<b>TOTAL PROJECT COST</b>", ParagraphStyle('TotalRow', parent=self.styles['Normal'], fontName='Helvetica-Bold')),
            Paragraph(f"<b>{formatted_cost}</b>", ParagraphStyle('TotalAmount', parent=self.styles['Normal'], fontName='Helvetica-Bold', alignment=TA_RIGHT)),
            Paragraph("<b>100.0%</b>", ParagraphStyle('TotalPercent', parent=self.styles['Normal'], fontName='Helvetica-Bold', alignment=TA_CENTER))
        ])
        
        cost_table = Table(cost_data, colWidths=[3.5*inch, 1.5*inch, 1*inch])
        cost_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 11),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('FONTSIZE', (0, 1), (-1, -2), 9),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('ROWBACKGROUNDS', (0, 1), (-1, -2), [colors.white, colors.HexColor('#f5f5f5')]),
            ('BACKGROUND', (0, -1), (-1, -1), colors.HexColor('#e5e7eb')),
            ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold'),
            ('FONTSIZE', (0, -1), (-1, -1), 10),
        ]))
        
        elements.append(cost_table)
        elements.append(Spacer(1, 20))
        
        # Enhanced Payment Schedule
        elements.append(Paragraph("Payment Schedule & Terms", ParagraphStyle(
            'FinSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=10
        )))
        
        # Format payment schedule better
        payment_lines = financial_proposal.payment_schedule.split('\n')
        formatted_payments = []
        for line in payment_lines:
            if line.strip() and ('Contract Signing' in line or 'Design Approval' in line or 'Development' in line or 'UAT' in line or 'Go-Live' in line):
                # Extract milestone and amount
                if ':' in line and '(' in line:
                    milestone = line.split(':')[0].replace('-', '').strip()
                    amount_part = line.split(':')[1].strip()
                    formatted_payments.append([milestone, amount_part])
        
        if formatted_payments:
            payment_data = [['Milestone', 'Payment Due']]
            for milestone, amount in formatted_payments:
                payment_data.append([milestone, amount])
            
            payment_table = Table(payment_data, colWidths=[3.5*inch, 2.5*inch])
            payment_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1e3a8a')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('FONTSIZE', (0, 0), (-1, -1), 9),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                ('ALIGN', (1, 1), (1, -1), 'RIGHT'),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f5f5f5')])
            ]))
            elements.append(payment_table)
        else:
            # Fallback to original format
            elements.append(Paragraph(financial_proposal.payment_schedule, self.styles['Normal']))
        
        elements.append(Spacer(1, 20))
        
        # Enhanced ROI Analysis
        elements.append(Paragraph("Return on Investment Analysis", ParagraphStyle(
            'FinSubheading',
            parent=self.styles['Heading3'],
            fontSize=14,
            textColor=colors.HexColor('#1e3a8a'),
            spaceAfter=10
        )))
        
        # Format ROI analysis with paragraph breaks
        roi_text = financial_proposal.roi_analysis
        if roi_text and len(roi_text) > 50:
            if "\n\n" in roi_text:
                paragraphs = roi_text.split('\n\n')
                for para in paragraphs:
                    if para.strip():
                        elements.append(Paragraph(para.strip(), self.styles['Normal']))
                        elements.append(Spacer(1, 8))
            else:
                elements.append(Paragraph(roi_text, self.styles['Normal']))
        else:
            # Enhanced fallback ROI analysis
            roi_fallback = """
            <b>Investment Returns & Business Value:</b><br/><br/>
            
            <b>Financial Benefits:</b><br/>
            • Expected ROI of 250-300% within 24 months<br/>
            • Operational cost reduction of 20-25%<br/>
            • Efficiency improvements of 30-40%<br/>
            • Reduced manual processing time by 50%<br/><br/>
            
            <b>Strategic Benefits:</b><br/>
            • Enhanced decision-making capabilities<br/>
            • Improved data accuracy and consistency<br/>
            • Scalable platform for future growth<br/>
            • Competitive advantage through automation<br/><br/>
            
            <b>Risk Mitigation:</b><br/>
            • Reduced dependency on manual processes<br/>
            • Improved compliance and audit capabilities<br/>
            • Enhanced security and data protection
            """
            elements.append(Paragraph(roi_fallback, self.styles['Normal']))
        
        elements.append(Spacer(1, 15))
        
        # Value Proposition Summary
        value_prop = """
        <b>Why Choose Abougia Technologies:</b><br/>
        • Proven track record with similar implementations<br/>
        • Fixed-price model eliminates budget risks<br/>
        • Comprehensive warranty and support included<br/>
        • Competitive pricing with superior value delivery<br/>
        • Dedicated project team with relevant expertise
        """
        
        elements.append(Paragraph(value_prop, ParagraphStyle(
            'ValueProp',
            parent=self.styles['Normal'],
            fontSize=10,
            borderWidth=1,
            borderColor=colors.HexColor('#059669'),
            borderPadding=12,
            backColor=colors.HexColor('#f0fdf4')
        )))
        
        elements.append(PageBreak())
        return elements
    
    def generate_rfp_response_document(self, rfp_response, output_path: str):
        """Generate complete RFP response document"""
        doc = SimpleDocTemplate(
            output_path,
            pagesize=A4,
            topMargin=0.75*inch,
            bottomMargin=0.75*inch,
            leftMargin=0.75*inch,
            rightMargin=0.75*inch
        )
        
        elements = []
        
        # Cover page
        elements.extend(self.create_cover_page(rfp_response))
        
        # Table of contents
        elements.append(Paragraph("TABLE OF CONTENTS", self.section_style))
        elements.append(Spacer(1, 15))
        
        toc_items = [
            "1. Executive Summary",
            "2. Technical Proposal",
            "3. Financial Proposal", 
            "4. Project Timeline",
            "5. Risk Mitigation",
            "6. Team Composition",
            "7. Compliance Statement",
            "8. References & Case Studies",
            "9. Assumptions & Dependencies"
        ]
        
        for item in toc_items:
            elements.append(Paragraph(item, self.styles['Normal']))
            elements.append(Spacer(1, 6))
        
        elements.append(PageBreak())
        
        # Main sections
        elements.extend(self.create_executive_summary_section(rfp_response))
        elements.extend(self.create_technical_section(rfp_response.technical_proposal))
        elements.extend(self.create_financial_section(rfp_response.financial_proposal))
        
        # Project Timeline
        elements.append(Paragraph("PROJECT TIMELINE", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Format timeline with better structure
        timeline_text = rfp_response.project_timeline
        timeline_lines = timeline_text.split('\n')
        
        for line in timeline_lines:
            line = line.strip()
            if line:
                if line.startswith('Project Timeline:'):
                    # Main timeline header
                    elements.append(Paragraph(line, ParagraphStyle(
                        'TimelineHeader',
                        parent=self.styles['Normal'],
                        fontName='Helvetica-Bold',
                        fontSize=12,
                        spaceAfter=10
                    )))
                elif line.startswith('Phase'):
                    # Phase headers
                    elements.append(Paragraph(line, ParagraphStyle(
                        'PhaseHeader',
                        parent=self.styles['Normal'],
                        fontName='Helvetica-Bold',
                        fontSize=11,
                        spaceAfter=6,
                        textColor=colors.HexColor('#1e3a8a')
                    )))
                elif line.startswith('-'):
                    # Phase activities
                    elements.append(Paragraph(line, ParagraphStyle(
                        'PhaseActivity',
                        parent=self.styles['Normal'],
                        fontSize=9,
                        leftIndent=20,
                        spaceAfter=2
                    )))
                elif line.startswith('Parallel Activities:'):
                    # Parallel activities header
                    elements.append(Spacer(1, 8))
                    elements.append(Paragraph(line, ParagraphStyle(
                        'ParallelHeader',
                        parent=self.styles['Normal'],
                        fontName='Helvetica-Bold',
                        fontSize=10,
                        spaceAfter=4
                    )))
                else:
                    # Regular text
                    elements.append(Paragraph(line, self.styles['Normal']))
                    elements.append(Spacer(1, 4))
        
        elements.append(PageBreak())
        
        # Risk Mitigation
        elements.append(Paragraph("RISK MITIGATION", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Format risk mitigation with paragraph breaks
        risk_text = rfp_response.risk_mitigation
        if "\n\n" in risk_text:
            paragraphs = risk_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    # Check if it's a header or regular text
                    if para.strip().endswith(':'):
                        elements.append(Paragraph(para.strip(), ParagraphStyle(
                            'RiskHeader',
                            parent=self.styles['Normal'],
                            fontName='Helvetica-Bold',
                            fontSize=11,
                            spaceAfter=6
                        )))
                    else:
                        elements.append(Paragraph(para.strip(), self.styles['Normal']))
                        elements.append(Spacer(1, 8))
        else:
            elements.append(Paragraph(risk_text, self.styles['Normal']))
        
        elements.append(Spacer(1, 20))
        
        # Team Composition
        elements.append(Paragraph("TEAM COMPOSITION", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Format team composition with better structure
        team_text = rfp_response.team_composition
        team_lines = team_text.split('\n')
        
        for line in team_lines:
            line = line.strip()
            if line:
                if line.endswith(':'):
                    # Section headers
                    elements.append(Paragraph(line, ParagraphStyle(
                        'TeamHeader',
                        parent=self.styles['Normal'],
                        fontName='Helvetica-Bold',
                        fontSize=11,
                        spaceAfter=6,
                        textColor=colors.HexColor('#1e3a8a')
                    )))
                elif line.startswith('-'):
                    # Team member roles
                    elements.append(Paragraph(line, ParagraphStyle(
                        'TeamMember',
                        parent=self.styles['Normal'],
                        fontSize=9,
                        leftIndent=20,
                        spaceAfter=3
                    )))
                else:
                    # Regular text
                    elements.append(Paragraph(line, self.styles['Normal']))
                    elements.append(Spacer(1, 4))
        
        elements.append(PageBreak())
        
        # Compliance
        elements.append(Paragraph("COMPLIANCE STATEMENT", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Format compliance with paragraph breaks
        compliance_text = rfp_response.compliance_statement
        if "\n\n" in compliance_text:
            paragraphs = compliance_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    if para.strip().endswith(':'):
                        elements.append(Paragraph(para.strip(), ParagraphStyle(
                            'ComplianceHeader',
                            parent=self.styles['Normal'],
                            fontName='Helvetica-Bold',
                            fontSize=11,
                            spaceAfter=6
                        )))
                    else:
                        elements.append(Paragraph(para.strip(), self.styles['Normal']))
                        elements.append(Spacer(1, 8))
        else:
            elements.append(Paragraph(compliance_text, self.styles['Normal']))
        
        elements.append(Spacer(1, 20))
        
        # References
        elements.append(Paragraph("REFERENCES & CASE STUDIES", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Format references with paragraph breaks
        references_text = rfp_response.references_case_studies
        if "\n\n" in references_text:
            paragraphs = references_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    elements.append(Paragraph(para.strip(), self.styles['Normal']))
                    elements.append(Spacer(1, 8))
        else:
            elements.append(Paragraph(references_text, self.styles['Normal']))
        
        elements.append(Spacer(1, 20))
        
        # Assumptions
        elements.append(Paragraph("ASSUMPTIONS & DEPENDENCIES", self.section_style))
        elements.append(Spacer(1, 15))
        
        # Format assumptions with better structure
        assumptions_text = rfp_response.assumptions_dependencies
        assumptions_lines = assumptions_text.split('\n')
        
        for line in assumptions_lines:
            line = line.strip()
            if line:
                if line.endswith(':'):
                    # Section headers
                    elements.append(Paragraph(line, ParagraphStyle(
                        'AssumptionHeader',
                        parent=self.styles['Normal'],
                        fontName='Helvetica-Bold',
                        fontSize=11,
                        spaceAfter=6,
                        textColor=colors.HexColor('#1e3a8a')
                    )))
                elif line.startswith('-'):
                    # Assumption items
                    elements.append(Paragraph(line, ParagraphStyle(
                        'AssumptionItem',
                        parent=self.styles['Normal'],
                        fontSize=9,
                        leftIndent=20,
                        spaceAfter=3
                    )))
                else:
                    # Regular text
                    elements.append(Paragraph(line, self.styles['Normal']))
                    elements.append(Spacer(1, 4))
        
        # Build document
        try:
            doc.build(elements)
            print(f"✅ RFP Response generated successfully: {output_path}")
        except Exception as e:
            print(f"❌ Error generating RFP response: {e}")
            import traceback
            traceback.print_exc()

In [180]:
# Enhanced main function to generate both feasibility and response
def generate_rfp_response():
    """Generate complete RFP response after feasibility analysis"""
    
    print("\n" + "="*70)
    print("GENERATING RFP RESPONSE...")
    print("="*70)
    
    # Initialize analyzer (reuse from existing code)
    analyzer = RFPFeasibilityAnalyzer()
    
    # File paths (same as your existing code)
    file_paths = {
        'module_matching': '/kaggle/input/input-reports-fr-agent/module-matching-report.pdf',
        'win_loss': '/kaggle/input/input-reports-fr-agent/win_loss_analysis.pdf', 
        'gap_analysis': '/kaggle/input/input-reports-fr-agent/gap_analysis_report.pdf',
        'customer_needs': '/kaggle/input/input-reports-fr-agent/customer_needs_report.pdf'
    }
    
    try:
        # Load and analyze documents
        print("📄 Loading and analyzing documents...")
        results = analyzer.load_and_analyze_documents(file_paths)
        
        # Calculate feasibility
        print("🔍 Calculating feasibility...")
        assessment = analyzer.calculate_feasibility()
        
        # Only generate response if feasible
        if assessment.can_respond:
            print("✅ Project is feasible - generating RFP response...")
            
            # Initialize response generator
            response_generator = RFPResponseGenerator(analyzer.processor.llm_processor)
            
            # Generate comprehensive response
            print("🤖 Generating technical and financial proposals...")
            rfp_response = response_generator.generate_comprehensive_response(analyzer, assessment)
            
            # Generate response document
            response_output_path = "/kaggle/working/rfp_response_proposal.pdf"
            print("📝 Creating RFP response document...")
            
            response_doc_generator = RFPResponseReportGenerator()
            response_doc_generator.generate_rfp_response_document(rfp_response, response_output_path)
            
            # Display response summary
            print("\n" + "="*70)
            print("RFP RESPONSE SUMMARY")
            print("="*70)
            print(f"💰 Total Investment: {rfp_response.financial_proposal.total_cost}")
            print(f"⏱️  Project Duration: {rfp_response.project_timeline.split('weeks total')[0].split(':')[1].strip()} weeks")
            print(f"👥 Team Size: Core team + specialists")
            print(f"🎯 Key Modules: {len([m for m in analyzer.modules if m.status == 'Available'])} available, {len([m for m in analyzer.modules if m.status == 'Missing'])} to develop")
            
            # Show cost breakdown summary
            print("\n💳 COST BREAKDOWN SUMMARY:")
            for component, cost in list(rfp_response.financial_proposal.cost_breakdown.items())[:5]:
                print(f"   • {component}: {cost}")
            
            # Show key technical highlights
            print("\n🔧 TECHNICAL HIGHLIGHTS:")
            print(f"   • Solution: {rfp_response.technical_proposal.solution_overview[:100]}...")
            
            print(f"\n📊 Response document saved to: {response_output_path}")
            
        else:
            print("❌ Project not feasible - RFP response not recommended")
            print(f"   Confidence Score: {assessment.confidence_score:.1f}%")
            print(f"   Critical Gaps: {len(assessment.critical_gaps)}")
            print("   Consider addressing gaps before proceeding")
            
    except Exception as e:
        print(f"❌ Error generating RFP response: {e}")

In [181]:
def main():
    """Main function to run the RFP feasibility analysis"""
    
    print("=" * 70)
    print("RFP RESPONSE FEASIBILITY ANALYZER v2.1")
    print("=" * 70)
    print("Comprehensive document analysis for RFP go/no-go decision making")
    print("-" * 70)
    
    # Initialize the analyzer
    analyzer = RFPFeasibilityAnalyzer()
    
    # Define file paths
    file_paths = {
        'module_matching': '/kaggle/input/input-reports-fr-agent/module-matching-report.pdf',
        'win_loss': '/kaggle/input/input-reports-fr-agent/win_loss_analysis.pdf', 
        'gap_analysis': '/kaggle/input/input-reports-fr-agent/gap_analysis_report.pdf',
        'customer_needs': '/kaggle/input/input-reports-fr-agent/customer_needs_report.pdf'
    }
    
    print("\n📁 Input Documents:")
    for doc_type, path in file_paths.items():
        print(f"   • {doc_type.replace('_', ' ').title()}: {os.path.basename(path)}")
    
    print("\n" + "-" * 70)
    print("STARTING ANALYSIS...")
    print("-" * 70)
    
    try:
        # Load and analyze documents
        results = analyzer.load_and_analyze_documents(file_paths)
        
        print("\n" + "=" * 70)
        print("EXTRACTION SUMMARY")
        print("=" * 70)
        print(f"Documents Processed: {results['documents_processed']}")
        print(f"Total Content Analyzed: {results['total_content_length']:,} characters")
        print("\nExtracted Items:")
        for item_type, count in results['extraction_summary'].items():
            print(f"   • {item_type.replace('_', ' ').title()}: {count}")
        
        # Display extracted modules
        print("\nModules Identified:")
        if analyzer.modules:
            for module in analyzer.modules:
                print(f"   • {module.module_name} (Status: {module.status}, Priority: {module.client_priority}, Coverage: {module.coverage_percentage:.0f}%)")
        else:
            print("   • No modules identified (fallback to default module may apply)")
        
        # Display extracted gaps with structured format
        print("\nGaps Identified:")
        if analyzer.gaps:
            for gap in analyzer.gaps:
                print(f"   • {gap.description}")
                print(f"     Type: {gap.gap_type}, Severity: {gap.severity}")
        else:
            print("   • No gaps identified (fallback to default gap may apply)")
        
        # Calculate feasibility
        print("\n" + "-" * 70)
        print("CALCULATING FEASIBILITY...")
        print("-" * 70)
        
        assessment = analyzer.calculate_feasibility()
        
        # Display results
        print("\n" + "=" * 70)
        print("FEASIBILITY ASSESSMENT RESULTS")
        print("=" * 70)
        print(f"\n{'✅ GO' if assessment.can_respond else '❌ NO-GO'} - {'Proceed with RFP Response' if assessment.can_respond else 'Do Not Proceed'}")
        print(f"\nConfidence Score: {assessment.confidence_score:.1f}%")
        print(f"Win Probability: {assessment.win_probability:.1f}%")
        print(f"\nModule Coverage: {assessment.available_modules}/{assessment.total_modules} ({(assessment.available_modules/assessment.total_modules*100) if assessment.total_modules > 0 else 0:.0f}%)")
        print(f"Critical Gaps: {len(assessment.critical_gaps)}")
        print(f"Total Risks: {len(assessment.risks)}")
        
        # Detailed gap summary
        if assessment.critical_gaps:
            print("\nCritical Gaps (High/Critical Severity):")
            for gap in assessment.critical_gaps:
                print(f"   • {gap.description}")
                print(f"     Type: {gap.gap_type}, Severity: {gap.severity}, Mitigation: {gap.mitigation_strategy}")
        
        # Investment summary
        print(f"\nInvestment Required: {assessment.investment_required}")
        print(f"Timeline Estimate: {assessment.timeline_estimate}")
        print(f"Resources Needed: {assessment.resource_requirements}")
        
        # Top actions
        if assessment.required_actions:
            print("\n🎯 TOP PRIORITY ACTIONS:")
            for i, (action, priority, timeline, owner) in enumerate(assessment.required_actions[:3], 1):
                print(f"   {i}. [{priority}] {action}")
                print(f"      Timeline: {timeline} | Owner: {owner}")
        
        # Generate PDF report
        output_path = "/kaggle/working/rfp_feasibility_report_enhanced.pdf"
        print("\n" + "-" * 70)
        print("GENERATING COMPREHENSIVE REPORT...")
        print("-" * 70)
        
        generator = RFPFeasibilityReportGenerator()
        generator.generate_report(analyzer, assessment, output_path)
        
        print("\n" + "=" * 70)
        print("✅ ANALYSIS COMPLETE!")
        print("=" * 70)
        print(f"📊 Detailed report saved to: {output_path}")

        
    except Exception as e:
        print(f"\n❌ ERROR during analysis: {e}")
        print("Please verify that all input files exist and are readable at the specified paths:")
        for doc_type, path in file_paths.items():
            print(f"   • {doc_type.replace('_', ' ').title()}: {path}")


In [182]:
def main_enhanced():
    """Enhanced main function that generates both feasibility and response"""
    
    print("="*70)
    print("RFP ANALYSIS & RESPONSE GENERATOR v2.1")
    print("="*70)
    print("Complete solution: Feasibility Analysis + RFP Response")
    print("-"*70)
    
    # Run original feasibility analysis
    main()
    
    # Generate RFP response if feasible
    generate_rfp_response()
    
    print("\n" + "="*70)
    print("✅ COMPLETE ANALYSIS & RESPONSE GENERATION FINISHED!")
    print("="*70)
    print("Generated files:")
    print("   📊 Feasibility Report: /kaggle/working/rfp_feasibility_report_enhanced.pdf")
    print("   📝 RFP Response: /kaggle/working/rfp_response_proposal.pdf")

# Alternative standalone function for just RFP response generation
def standalone_rfp_response():
    """Standalone function to generate RFP response without feasibility check"""
    
    print("="*70)
    print("STANDALONE RFP RESPONSE GENERATOR")
    print("="*70)
    
    analyzer = RFPFeasibilityAnalyzer()
    
    file_paths = {
        'module_matching': '/kaggle/input/input-reports-fr-agent/module-matching-report.pdf',
        'win_loss': '/kaggle/input/input-reports-fr-agent/win_loss_analysis.pdf', 
        'gap_analysis': '/kaggle/input/input-reports-fr-agent/gap_analysis_report.pdf',
        'customer_needs': '/kaggle/input/input-reports-fr-agent/customer_needs_report.pdf'
    }
    
    try:
        # Load documents
        results = analyzer.load_and_analyze_documents(file_paths)
        assessment = analyzer.calculate_feasibility()
        
        # Force generate response regardless of feasibility
        print("🚀 Generating RFP response (bypassing feasibility check)...")
        
        response_generator = RFPResponseGenerator(analyzer.processor.llm_processor)
        rfp_response = response_generator.generate_comprehensive_response(analyzer, assessment)
        
        # Generate documents
        response_output_path = "/kaggle/working/rfp_response_standalone.pdf"
        response_doc_generator = RFPResponseReportGenerator()
        response_doc_generator.generate_rfp_response_document(rfp_response, response_output_path)
        
        print(f"✅ Standalone RFP response generated: {response_output_path}")
        
        # Show financial summary
        print("\n💰 FINANCIAL SUMMARY:")
        print(f"   Total Cost: {rfp_response.financial_proposal.total_cost}")
        print(f"   Payment Terms: Milestone-based with 20% upfront")
        print(f"   ROI Expected: Positive within 24 months")
        
    except Exception as e:
        print(f"❌ Error in standalone generation: {e}")

# Usage examples:
# 1. Run complete analysis + response: main_enhanced()
# 2. Run just feasibility: main() 
# 3. Run just RFP response: standalone_rfp_response()
# 4. Run targeted response: generate_rfp_response()

if __name__ == "__main__":
    # Use this for complete analysis and response generation
    standalone_rfp_response()

STANDALONE RFP RESPONSE GENERATOR


Device set to use cuda:0


Initialized LLMProcessor with TinyLlama/TinyLlama-1.1B-Chat-v1.0 on cuda

📄 Analyzing module_matching...
  ✓ Extracted 17127 characters
  🤖 Running AI analysis...
  ✓ AI insights generated
  ✓ Extracted 13 modules

📄 Analyzing win_loss...
  ✓ Extracted 11671 characters
  🤖 Running AI analysis...
  ✓ AI insights generated
  ✓ Extracted 22 win/loss factors

📄 Analyzing gap_analysis...
  ✓ Extracted 3812 characters
  🤖 Running AI analysis...
  ✓ AI insights generated
  ✓ Extracted 4 gaps

📄 Analyzing customer_needs...
  ✓ Extracted 2801 characters
  🤖 Running AI analysis...
  ✓ AI insights generated
  ✓ Extracted 4 requirements

🤖 Generating AI insights...
🚀 Generating RFP response (bypassing feasibility check)...
❌ Error in standalone generation: 'RFPResponseGenerator' object has no attribute 'generate_comprehensive_response'
